# 0. Importación de librerias

In [1]:
# Librerías para el análisis de datos
import numpy as np  # Biblioteca para operaciones numéricas, especialmente útil para trabajar con matrices y arrays
import pandas as pd  # Biblioteca para manipulación y análisis de datos, especialmente para DataFrames
import statsmodels.api as sm  # Biblioteca para realizar modelos estadísticos, útil para regresiones y otros análisis estadísticos
from collections import Counter  # Contador de elementos en colecciones, útil para contar la frecuencia de elementos en una lista o array

# Librería para optimización
from gurobipy import Model, GRB, quicksum  # Gurobi es una biblioteca de optimización matemática. 
# Model permite definir el modelo de optimización,
# GRB contiene constantes (como tipos de variables y sentido de optimización),
# quicksum permite realizar sumas de forma rápida y eficiente en Gurobi.

# Librerías operacionales
import os  # Biblioteca para interactuar con el sistema operativo (ej. manejo de archivos y rutas)
from datetime import datetime  # Módulo para trabajar con fechas y tiempos, útil para capturar fechas actuales o manipular datos de tiempo


# <h2>1. Carga de datos desde excel a dataframes de pandas</h2>


En primer lugar se cargan los datos iniciales desde los csv que nos proveen. (Esto puede tardar un ratito)

In [2]:
url_tratamiento = 'https://drive.usercontent.google.com/download?id=1KTRwYGaWoQQnZwbk8VpdxKULMaOReoF5&authuser=0&confirm=t&uuid=4e4d7983-c5a5-412d-9f5b-b9bda1068b73&at=AENtkXZSxkDr84kWrDlz6ANq4ov2%3A1730951312566'

df_tratamiento = pd.read_csv("Tratamiento.csv")

In [3]:
file_id = '173bRZQG7NWfdpHJ-o4-NA3ieH1-Fhyok'
url_informacion_de_clientes = f'https://drive.google.com/uc?id={file_id}'

df_informacion_de_clientes = pd.read_csv("Informacion_Clientes.csv")

In [4]:
url_simulaciones_clientes = 'https://drive.usercontent.google.com/download?id=1IXyKwtKFLCUsAV1MtNqktiwKaHzV2D5A&authuser=0&confirm=t&uuid=edd22376-238f-4c1f-b603-728b07bafd7f&at=AENtkXaEURpV52p_BdWxyisvjhSQ%3A1730951026384'

df_simulaciones_clientes = pd.read_csv("Simulaciones_clientes.csv")

In [5]:
file_id = '1Z4jMzZeD2q-4-ioSgyimppAdLZzqDkt3'
url_ventas = f'https://drive.google.com/uc?id={file_id}'

df_ventas = pd.read_csv("Ventas.csv")

## <h3>1.1 Carga de 'Informacion_Clientes.csv'</h3>

En primer lugar se cargará la información de los clientes. Esto incluye las siguientes características de los clientes:

* **unnamed**: algo como uid
* **Rut**: identificador de Chile (supongo que por privacidad va desde 0 a max de observaciones)
* **Género**: Masculino o femenino
* **Categoría_Digital**: Si el cliente es digital o no
* **Elasticidad_Precios**: Baja, media o alta
* **Nacionalidad**: Chileno o extranjero
* **Propensión**: Número entre 0 y 1 que idica que tan propenso a cursar un credito es el cliente
* **Probabilidad_No_Pago**: Número entre 0 y 1 que indica la probabilidad de que el cliente no pague la deuda
* **Edad**: Numero entero de edad en años
* **Renta**: Renta promedio de los últimos 12 meses
* **Oferta_Consumo**: Monto máximo que puede cursar un cliente dado sus antecedentes crediticios y situación socioeconómica. 
* **Deuda_CMF**: Deuda que tiene el cliente en otros bancos. Efectivamente es deuda pendiente, pero de créditos otorgados por la competencia.
* **Tiempo_como_cliente**: Número de tiempo(no sé en que medida está) que el cliente lleva en el banco


Se elimina el tiempo como cliente ya que no aporta información

In [6]:
df_informacion_de_clientes.drop(columns=['Tiempo_como_cliente'], inplace=True)

## <h3>1.2 Carga de 'Simulaciones_Clientes.csv'</h3>



En segundo lugar se cargaran las simulaciones hechas por los clientes en la página del banco. Esto incluye las siguientes características de las simulaciones:
* **unnamed**: Supongo que es el número de simulacion registrada, un tipo de identificador de la simulación
* **fecha**: yyyy-mm-dd fecha de la simulación
* **rut**: identificador de Chile del cliente que hizo la simulacion
* **monto_simulado**: monto prestado al cliente
* **plazo_simulado**: plazo en **meses** del crédito
* **tasa_simulado**: costo para el cliente del credito

In [7]:
df_simulaciones_clientes = df_simulaciones_clientes[df_simulaciones_clientes['Monto_Simulado'] > 0]

## <h3>1.3 Carga de 'Tratamiento.csv'</h3>

En tercer lugar se cargara el tratamiento que ha tenido el banco con el cliente, es decir, cómo se han contactado con él. Esto incluye las siguientes características:

* **unnamed**: Número de tratamiento registrado
* **fecha**: yyyy-mm-dd
* **rut**: Identificador de Chile del cliente con el que se tiene el tipo de trato
* **n_correos**: Cantidad de correos que se enviaron en el mes que sale la fecha. Es decir, si sele fecha '2024-03-01', correspondería a los correos enviados en marzo de 2024.
* **asg_ejec**: Si el cliente tiene un ejecutivo asignado

## <h3>1.4 Carga de 'Ventas.csv'</h3>

Por último se cargaran las ventas que ha tenido el banco con el cliente. Esto incluye las siguientes características:

* **unnamed**: Índice sin significado
* **fecha**: yyyy-mm-dd -> fecha en la que se concretó la venta
* **rut**: identificador de Chile del cliente al que se le concretó la venta

# <h2>2. Joints de datos<h2>

In [8]:
# Unir los DataFrames 'df_informacion_de_clientes' y 'df_simulaciones_clientes' en base a la columna 'rut'
# El método 'how="left"' asegura que todos los registros de 'df_informacion_de_clientes' se conserven,
# incluso si no tienen coincidencia en 'df_simulaciones_clientes'.
df_simulaciones_e_informacion_de_clientes = pd.merge(
    df_informacion_de_clientes, 
    df_simulaciones_clientes, 
    on='rut', 
    how='left'
)

# Crear una nueva columna 'simulo' que indica si el cliente tiene un 'Monto_Simulado' o no
# El método 'notna()' devuelve True para valores no nulos y False para nulos.
# Luego, 'astype(int)' convierte estos valores booleanos en enteros (1 para True, 0 para False).
df_simulaciones_e_informacion_de_clientes['simulo'] = df_simulaciones_e_informacion_de_clientes['Monto_Simulado'].notna().astype(int)

# Eliminar columnas innecesarias 'Unnamed: 0_x' y 'Unnamed: 0_y' que podrían haber surgido durante la carga o manipulación de datos
df_simulaciones_e_informacion_de_clientes.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)


In [9]:
df_simulaciones_e_informacion_de_clientes

,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,fecha,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,simulo
0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2019-10-01,319936.0,33.0,1.092295,1
1,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2019-11-01,249773.0,30.0,2.324675,1
2,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2020-02-01,280087.0,28.0,1.051704,1
3,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2021-05-01,289780.0,26.0,2.193118,1
4,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2021-06-01,258061.0,22.0,2.188368,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8517080,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2023-03-01,403331.0,17.0,2.356028,1
8517081,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2023-08-01,359897.0,16.0,2.478376,1
8517082,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2024-03-01,348048.0,18.0,2.301079,1
8517083,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2024-05-01,344504.0,16.0,2.462272,1


In [10]:
# Unir los DataFrames 'df_simulaciones_e_informacion_de_clientes' y 'df_ventas' en base a las columnas 'rut' y 'fecha'
# El método 'how="left"' asegura que todos los registros de 'df_simulaciones_e_informacion_de_clientes' se conserven,
# incluso si no tienen coincidencia en 'df_ventas'.
df_simulaciones_e_informacion_de_clientes_ventas = pd.merge( 
    df_simulaciones_e_informacion_de_clientes, 
    df_ventas, 
    on=['rut', 'fecha'], 
    how='left'
)

# Crear una nueva columna 'venta' que indica si existe una venta asociada al cliente y la fecha específica
# El método 'notna()' verifica si hay un valor no nulo en la columna 'Unnamed: 0' (que indica presencia de una venta)
# Luego, 'astype(int)' convierte estos valores booleanos en enteros (1 para True, 0 para False).
df_simulaciones_e_informacion_de_clientes_ventas['venta'] = df_simulaciones_e_informacion_de_clientes_ventas['Unnamed: 0'].notna().astype(int)


In [11]:
# Unir los DataFrames 'df_simulaciones_e_informacion_de_clientes_ventas' y 'df_tratamiento' en base a las columnas 'rut' y 'fecha'
# La unión se realiza con 'how="left"' para conservar todos los registros de 'df_simulaciones_e_informacion_de_clientes_ventas'
# incluso si no tienen coincidencia en 'df_tratamiento'.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento = pd.merge( 
    df_simulaciones_e_informacion_de_clientes_ventas, 
    df_tratamiento, 
    on=['rut', 'fecha'], 
    how='left'
)

# Crear una nueva columna 'mes' que extrae el mes y año de la columna 'fecha'
# Primero se convierte 'fecha' al formato datetime, luego 'dt.to_period('M')' obtiene el periodo del mes/año.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['mes'] = pd.to_datetime(df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['fecha']).dt.to_period('M')

# Eliminar las columnas 'Unnamed: 0_x' y 'Unnamed: 0_y' ya que no aportan información relevante
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)

# 3. CLUSTERING POR POLITICAS

### Seteo de cluster. Aquí se definen las variables y sus cortes. La idea es que el algoritmo de RL haga sus acciones en esta sección

Aquí se definen que variables se utilizarán para crear el cluster

In [12]:
df_informacion_de_clientes_procesados_cluster_definitivo = df_informacion_de_clientes[['rut', 'Categoria_Digital', 'Edad', 'Genero', 'Renta', 'Propension', 'Probabilidad_No_Pago']].copy()


Aqui se definen en que partes y en cuantas partes se particionarán las variables escogidas anteriormente

In [13]:
# Crear una copia del DataFrame 'df_informacion_de_clientes_procesados_cluster_definitivo' para trabajar sin modificar el original
df = df_informacion_de_clientes_procesados_cluster_definitivo.copy()

# Clasificar la columna 'Propension' en tres categorías usando cuantiles, lo que permite dividir los datos en terciles
# Las etiquetas indican si la propensión es baja, media o alta.
df['Categoria_Propenso'] = pd.qcut(df['Propension'], 3, labels=['Propension baja', 'Propension media', 'Propension alta'])

# Clasificar la columna 'Probabilidad_No_Pago' en cinco categorías, asignando etiquetas según los valores de probabilidad
# Cada categoría representa el nivel de confiabilidad en el pago: desde 'Muy buen pagador' hasta 'Muy mal pagador'.
df['Categoria_Probabilidad_No_Pago'] = pd.cut(df['Probabilidad_No_Pago'], 
                                              bins=[-float('inf'), 0.0011, 0.00149, 0.005, 0.006, float('inf')],
                                              labels=['Muy buen pagador', 'Buen pagador', 'Pagador neutro', 'Mal pagador', 'Muy mal pagador'])

# Clasificar la columna 'Edad' en tres categorías: 'Joven', 'Adulto' y 'Adulto Mayor'
# Cada categoría se define en función de rangos de edad especificados en 'bins'.
df['Categoria_Edad'] = pd.cut(df['Edad'], 
                              bins=[-float('inf'), 35, 60, float('inf')],
                              labels=['Joven', 'Adulto', 'Adulto Mayor'])

# Crear un DataFrame único de 'rut' y 'Renta' eliminando duplicados, para calcular percentiles de renta
df_unicos_renta = df[['rut', 'Renta']].drop_duplicates()

# Calcular el percentil de renta para cada cliente usando 'qcut' para dividir en 100 grupos
# Añadimos 1 para que los percentiles inicien en 1 en lugar de 0.
df_unicos_renta['Percentil_Renta'] = pd.qcut(df_unicos_renta['Renta'], 100, labels=False) + 1

# Clasificar la columna 'Percentil_Renta' en tres categorías: 'Renta Baja', 'Renta Media' y 'Renta Alta'
# Los rangos de percentil especificados en 'bins' definen estas categorías.
df_unicos_renta['Categoria_Renta'] = pd.cut(df_unicos_renta['Percentil_Renta'], 
                                            bins=[-float('inf'), 30, 80, float('inf')],
                                            labels=['Renta Baja', 'Renta Media', 'Renta Alta'])

# Incorporar la categoría de renta al DataFrame principal 'df' realizando una unión ('merge') en base a la columna 'rut'
df = df.merge(df_unicos_renta[['rut', 'Categoria_Renta']], on='rut', how='left')

# Mostrar el DataFrame resultante con las nuevas columnas creadas
df


,rut,Categoria_Digital,Edad,Genero,Renta,Propension,Probabilidad_No_Pago,Categoria_Propenso,Categoria_Probabilidad_No_Pago,Categoria_Edad,Categoria_Renta
0,1,Cliente no Digital,30.0,Masculino,6.258183e+05,0.997340,0.028445,Propension alta,Muy mal pagador,Joven,Renta Media
1,2,Cliente no Digital,41.0,Femenino,3.172616e+05,0.291601,0.014320,Propension baja,Muy mal pagador,Adulto,Renta Baja
2,3,Cliente no Digital,38.0,Femenino,1.240551e+07,0.685085,0.002156,Propension alta,Pagador neutro,Adulto,Renta Alta
3,4,Cliente no Digital,57.0,Masculino,5.441466e+05,0.914672,0.034418,Propension alta,Muy mal pagador,Adulto,Renta Baja
4,5,Cliente Digital,26.0,Masculino,1.870225e+05,0.425077,0.014978,Propension media,Muy mal pagador,Joven,Renta Baja
...,...,...,...,...,...,...,...,...,...,...,...
543646,543647,Cliente Digital,29.0,Femenino,1.176598e+05,0.144657,0.037291,Propension baja,Muy mal pagador,Joven,Renta Baja
543647,543648,Cliente no Digital,31.0,Masculino,1.558612e+06,0.740170,0.035877,Propension alta,Muy mal pagador,Joven,Renta Media
543648,543649,Cliente no Digital,49.0,Masculino,9.449508e+05,0.255285,0.023306,Propension baja,Muy mal pagador,Adulto,Renta Media
543649,543650,Cliente no Digital,40.0,Femenino,1.039964e+06,0.709086,0.015121,Propension alta,Muy mal pagador,Adulto,Renta Media


In [14]:
# Concatenar las variables especificadas en una nueva columna 'categoria_clusterizacion'
# La columna resultante combinará varias categorías en una descripción detallada del perfil del cliente.
# Convertimos cada columna a tipo string para asegurarnos de que los datos sean compatibles para la concatenación.

df['categoria_clusterizacion'] = (
    df['Categoria_Digital'].astype(str) + ' ' +              # Categoría de digitalización del cliente
    df['Categoria_Edad'].astype(str) + ' de genero ' +       # Categoría de edad, seguida de la palabra "de genero"
    df['Genero'].astype(str) + ' con ' +                     # Género del cliente
    df['Categoria_Propenso'].astype(str) + ' con una ' +     # Categoría de propensión, seguida de "con una"
    df['Categoria_Renta'].astype(str)                        # Categoría de renta
)


In [15]:
# Asignar un número único a cada entrada distinta en la columna 'categoria_clusterizacion'
# Se convierte la columna a tipo 'category', lo cual facilita la asignación de códigos numéricos únicos.
# 'cat.codes' asigna un código numérico único para cada valor único de 'categoria_clusterizacion'.
df['categoria_clusterizacion_numerica'] = df['categoria_clusterizacion'].astype('category').cat.codes


In [16]:
# Crear una copia del DataFrame con solo las columnas 'rut', 'categoria_clusterizacion' y 'categoria_clusterizacion_numerica'
# Esta copia se almacena en el nuevo DataFrame 'asignacion_clusters', el cual contendrá únicamente la identificación del cliente (rut),
# la descripción del perfil ('categoria_clusterizacion') y el código numérico asignado a cada perfil ('categoria_clusterizacion_numerica').
asignacion_clusters = df[['rut', 'categoria_clusterizacion', 'categoria_clusterizacion_numerica']].copy()


# 4. Estimacion de curvas de elasticidad por cluster

In [17]:
# Realizar una unión entre 'df_simulaciones_e_informacion_de_clientes_ventas' y 'asignacion_clusters' usando la columna 'rut' como clave
# Esta unión ('merge') se realiza con 'how="left"', lo que asegura que todos los registros de 'df_simulaciones_e_informacion_de_clientes_ventas' 
# se conserven, incluyendo aquellos sin coincidencia en 'asignacion_clusters'.
# La finalidad es agregar la información de clusterización (categoría y código numérico) al DataFrame de simulaciones y ventas.
df_estimar_elasticidad = pd.merge(df_simulaciones_e_informacion_de_clientes_ventas, asignacion_clusters, on='rut', how='left')


In [18]:
import os
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

def plot_elasticity_curve_with_histogram(tasas_grid, acceptance_probability, tasa_optima, df_cluster, cluster_num, output_folder):
    """
    Grafica la curva de elasticidad con la tasa óptima marcada y un histograma
    que muestra ventas y no ventas para cada precio, con mejoras en la visualización.
    
    Args:
    - tasas_grid (numpy array): Valores de la tasa simulada (eje X).
    - acceptance_probability (numpy array): Probabilidades de aceptación (eje Y).
    - tasa_optima (float): Tasa óptima encontrada.
    - df_cluster (DataFrame): Datos del cluster actual, incluyendo 'Tasa_Simulado' y 'venta'.
    - cluster_num (int): Número del cluster para el título.
    - output_folder (str): Ruta de la carpeta donde se guardará el gráfico.
    """
    # Preparar datos para el histograma
    ventas = df_cluster[df_cluster['venta'] == 1]['Tasa_Simulado']
    no_ventas = df_cluster[df_cluster['venta'] == 0]['Tasa_Simulado']
    
    # Crear la figura
    fig, ax1 = plt.subplots(figsize=(10, 6))
    
    # Curva de elasticidad
    ax1.plot(tasas_grid, acceptance_probability, label="Curva de Elasticidad", color='blue', linewidth=2)
    ax1.axvline(x=tasa_optima, color='green', linestyle='--', label=f"Tasa Óptima: {tasa_optima:.2f}%", zorder=10)
    ax1.set_xlabel("Tasa Simulada (%)", fontsize=12)
    ax1.set_ylabel("Probabilidad de Aceptación", fontsize=12, color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    ax1.set_ylim(0, 1)
    ax1.grid(True, linestyle='--', alpha=0.7)
    ax1.legend(loc='upper left', bbox_to_anchor=(1.2, 1))  # Leyenda fuera de la gráfica
    
    # Histograma de ventas y no ventas (usar el segundo eje Y)
    ax2 = ax1.twinx()
    ax2.hist([ventas, no_ventas], bins=20, color=['blue', 'red'], alpha=0.3, label=["Ventas", "No Ventas"], stacked=True)
    ax2.set_ylabel("Frecuencia de Ventas", fontsize=12, color='black')
    ax2.tick_params(axis='y', labelcolor='black')
    ax2.legend(loc='upper left', bbox_to_anchor=(1.2, 0.85))  # Leyenda fuera de la gráfica

    # Título
    plt.title(f"Curva de Elasticidad y Ventas por Tasa - Cluster {cluster_num}", fontsize=14)
    
    # Ajustar diseño para evitar superposiciones
    plt.tight_layout()
    
    # Guardar el gráfico
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    output_path = os.path.join(output_folder, f"curva_elasticidad_cluster_{cluster_num}.png")
    plt.savefig(output_path)
    print(f"Gráfico guardado en: {output_path}")
    plt.close()


### Este código realiza un análisis de elasticidad de ingresos en función de clusters de clientes. Primero, agrupa los datos por clusters definidos a través de variables de segmentación y filtra solo los datos relevantes para cada cluster. Luego, para cada cluster, se ajusta un modelo de regresión logística para predecir la probabilidad de aceptación de una simulación de crédito en función de la tasa de interés. A partir de este modelo, se crea una cuadrícula de tasas para estimar la probabilidad de aceptación y calcular el revenue potencial de cada simulación, teniendo en cuenta el monto medio simulado, el plazo medio simulado y la probabilidad media de no pago del cluster. Posteriormente, se determina la tasa que maximiza el revenue esperado y se calcula el número esperado de créditos aceptados, junto con el número de clientes únicos en cada cluster. Finalmente, los resultados se agregan tanto en listas globales como en un nuevo DataFrame, y luego se integran en el DataFrame original df_estimar_elasticidad, lo que permite analizar el revenue esperado total y otros indicadores clave en cada cluster.

In [19]:
def function_estimar_elasticidad(df_estimar_elasticidad):
    # Inicializar listas para almacenar resultados globales de revenue, clientes, créditos y simulaciones
    lista_revenue = []
    lista_clientes = []
    lista_creditos = []
    lista_simulaciones = []

    cluster_results = []  # Lista para almacenar resultados específicos de cada cluster

    # Obtener los números únicos de cada cluster
    cluster_numbers = df_estimar_elasticidad['categoria_clusterizacion_numerica'].unique()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_folder = f"hg_reglog_{timestamp}"
    os.makedirs(output_folder, exist_ok=True)

    # Iterar sobre cada cluster identificado por 'categoria_clusterizacion_numerica'
    for cluster_num in cluster_numbers:
        # Filtrar los datos correspondientes al cluster actual
        df_cluster = df_estimar_elasticidad[df_estimar_elasticidad['categoria_clusterizacion_numerica'] == cluster_num]
        
        # Asegurarse de que existen datos para ambos casos: venta == 1 y venta == 0
        if df_cluster.empty or df_cluster['venta'].isnull().all():
            continue  # Saltar este cluster si no cumple con la condición
        
        # Remover filas donde 'venta' o 'Tasa_Simulado' son nulos o infinitos
        df_cluster = df_cluster.replace([np.inf, -np.inf], np.nan)
        df_cluster = df_cluster.dropna(subset=['venta', 'Tasa_Simulado', 'Plazo_Simulado', 'Monto_Simulado', 'Probabilidad_No_Pago'])
        
        # Saltar el cluster si no hay suficientes puntos de datos
        if df_cluster.shape[0] < 10:
            continue
        
        # Extraer las variables 'venta' (como variable dependiente) y 'Tasa_Simulado' (como predictor)
        y = df_cluster['venta']
        X = df_cluster[['Tasa_Simulado']]
        
        # Añadir un término constante para el intercepto
        X = sm.add_constant(X)
        
        # Remover filas con valores NaN o Inf en X o y
        is_finite = np.isfinite(X).all(1) & np.isfinite(y)
        X = X[is_finite]
        y = y[is_finite]
        
        # Asegurarse de que después de remover NaN/Inf, todavía hay suficientes datos
        if len(y) < 10:
            continue
        
        # Ajustar el modelo de regresión logística
        logit_model = sm.Logit(y, X)
        try:
            result = logit_model.fit(disp=0)
        except:
            continue  # Saltar el cluster si el modelo no converge
        
        # Crear una cuadrícula de valores de 'Tasa_Simulado' para predicciones
        tasa_min = df_cluster['Tasa_Simulado'].min()
        tasa_max = df_cluster['Tasa_Simulado'].max()
        tasas_grid = np.linspace(tasa_min, tasa_max, 1000)
        
        # Predecir la probabilidad de aceptación usando el modelo ajustado
        X_grid = sm.add_constant(tasas_grid)
        acceptance_probability = result.predict(X_grid)
        
        # Asegurar que las probabilidades están en el rango [0, 1]
        acceptance_probability = np.clip(acceptance_probability, 0, 1)

        # Calcular valores medios necesarios para el cálculo de revenue
        n = df_cluster['Plazo_Simulado'].mean()
        vp = df_cluster['Monto_Simulado'].mean()
        pnp = df_cluster['Probabilidad_No_Pago'].mean()
        data = {
            'Plazo_Simulado_medio': n, 
            'Monto_Simulado_medio': vp, 
            'Probabilidad_No_Pago_media': pnp
        }
        
        # Calcular el revenue potencial
        i = tasas_grid / 100  # Convertir a decimal
        one_plus_i_pow_n = np.power(1 + i, n)
        annuity_factor = (i * one_plus_i_pow_n) / (one_plus_i_pow_n - 1)
        revenue = (n * vp * annuity_factor) - vp
        potential_revenue = revenue * (1 - pnp)
        
        # Calcular el promedio de simulaciones por fecha
        df_cluster_simulaciones_1 = df_cluster[df_cluster['simulo'] == 1]
        num_dates = df_cluster_simulaciones_1['fecha'].nunique()
        total_simulaciones = df_cluster_simulaciones_1['simulo'].sum()
        simulaciones_medias = total_simulaciones / num_dates if num_dates else 0
        
        # Saltar el cluster si no hay simulaciones
        if simulaciones_medias == 0:
            continue
        
        # Calcular el revenue esperado
        expected_revenue = acceptance_probability * potential_revenue * simulaciones_medias
        
        # Encontrar la tasa que maximiza el revenue esperado
        idx_max = np.argmax(expected_revenue)
        max_price = tasas_grid[idx_max]
        max_expected_revenue = expected_revenue[idx_max]
        
        # Probabilidad de aceptación en la tasa óptima
        prob_aceptacion_optima = acceptance_probability[idx_max]
        
        # Llamar a la función para graficar la curva y el histograma
        plot_elasticity_curve_with_histogram(
            tasas_grid, 
            acceptance_probability, 
            max_price, 
            df_cluster,  # Pasamos todo el DataFrame del cluster actual
            cluster_num, 
            output_folder
        )

        # Número esperado de créditos aceptados
        num_creditos_aceptados = round(prob_aceptacion_optima * simulaciones_medias)
        
        # Número de clientes únicos en el cluster
        num_clients = df_cluster['rut'].nunique()
        
        # Imprimir resultados para cada cluster
        print(f'Cluster {cluster_num}:')
        print(f'- Precio Máx. Revenue Esperado = {max_price:.2f}%')
        print(f'- Revenue Esperado Máximo = {max_expected_revenue:,.2f}')
        print(f'- Número de clientes en el cluster = {num_clients}')
        print(f'- Número de simulaciones en el cluster = {simulaciones_medias:.2f}')
        print(f'- Probabilidad de aceptación en el precio óptimo = {prob_aceptacion_optima:.4f}')
        print(f'- Número esperado de créditos aceptados = {num_creditos_aceptados}')
        print(f'- Monto medio simulado = {data["Monto_Simulado_medio"]:,.2f}')
        print(f'- Plazo medio simulado = {data["Plazo_Simulado_medio"]:,.2f}')
        print(f'- Probabilidad de no pago media = {data["Probabilidad_No_Pago_media"]:.4f}\n')

        # Agregar resultados a las listas globales
        lista_clientes.append(num_clients)
        lista_revenue.append(max_expected_revenue)
        lista_creditos.append(num_creditos_aceptados)
        lista_simulaciones.append(simulaciones_medias)
        
        # Almacenar resultados por cluster en cluster_results
        cluster_results.append({
            'categoria_clusterizacion_numerica': cluster_num,
            'tasa_optima': max_price,
            'probabilidad_aceptacion_optima': prob_aceptacion_optima,
            'revenue_esperado_maximo': max_expected_revenue,
            'numero_clientes': num_clients,
            'numero_simulaciones_medias': simulaciones_medias,
            'numero_creditos_esperados': num_creditos_aceptados,
            'monto_medio_simulado': data["Monto_Simulado_medio"],
            'plazo_medio_simulado': data["Plazo_Simulado_medio"],
            'probabilidad_no_pago_media': data["Probabilidad_No_Pago_media"]
        })

    # Imprimir resultados globales
    total_revenue = sum(lista_revenue)
    total_clientes = sum(lista_clientes)
    total_simulaciones = sum(lista_simulaciones)
    total_creditos = sum(lista_creditos)

    print(f"El revenue total esperado es: {total_revenue:,.2f} con un total de {total_clientes} clientes, "
        f"{total_simulaciones:,.2f} simulaciones, y {total_creditos} créditos.")

    # Crear un DataFrame a partir de cluster_results
    df_cluster_results = pd.DataFrame(cluster_results)

    # Incorporar los resultados por cluster de 'df_cluster_results' a 'df_estimar_elasticidad'
    df_estimar_elasticidad = df_estimar_elasticidad.merge(
        df_cluster_results[['categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']],
        on='categoria_clusterizacion_numerica', 
        how='left'
    )
    return {'df_estimar_elasticidad': df_estimar_elasticidad, 'total_revenue': total_revenue, 'total_clientes': total_clientes, 'total_simulaciones': total_simulaciones, 'total_creditos': total_creditos}


In [20]:
df_estimar_elasticidad = function_estimar_elasticidad(df_estimar_elasticidad)['df_estimar_elasticidad']

Gráfico guardado en: hg_reglog_20241127_184024\curva_elasticidad_cluster_101.png
Cluster 101:
- Precio Máx. Revenue Esperado = 1.26%
- Revenue Esperado Máximo = 446,196,158.99
- Número de clientes en el cluster = 6422
- Número de simulaciones en el cluster = 1814.08
- Probabilidad de aceptación en el precio óptimo = 0.4998
- Número esperado de créditos aceptados = 907
- Monto medio simulado = 2,672,066.39
- Plazo medio simulado = 27.47
- Probabilidad de no pago media = 0.0276

Gráfico guardado en: hg_reglog_20241127_184024\curva_elasticidad_cluster_76.png
Cluster 76:
- Precio Máx. Revenue Esperado = 1.22%
- Revenue Esperado Máximo = 65,495,606.51
- Número de clientes en el cluster = 5907
- Número de simulaciones en el cluster = 912.80
- Probabilidad de aceptación en el precio óptimo = 0.5523
- Número esperado de créditos aceptados = 504
- Monto medio simulado = 712,403.67
- Plazo medio simulado = 27.65
- Probabilidad de no pago media = 0.0092

Gráfico guardado en: hg_reglog_20241127_18

# 5. Estimacion de respuesta a tratamiento por cluster

In [21]:
# Unir los DataFrames 'df_tratamiento' y 'df_simulaciones_clientes' usando las columnas 'rut' y 'fecha' como claves
# La unión se realiza con 'how="left"', lo cual asegura que todos los registros de 'df_tratamiento' se conserven,
# incluyendo aquellos sin coincidencia en 'df_simulaciones_clientes'.
# Esta operación permite combinar la información de tratamiento con los datos de simulaciones de clientes.
df_simulaciones_info = pd.merge(df_tratamiento, df_simulaciones_clientes, on=['rut', 'fecha'], how='left')


In [22]:
# Usar operaciones de cadenas vectorizadas para crear la columna 'Tratamiento'
# Esta columna concatenará información sobre el ejecutivo asignado y el número de correos enviados.
# Se convierte 'asg_ejec' a string para poder concatenar, y 'n_correos' se convierte primero a entero y luego a string.
# El formato final es: "Ejecutivo=<valor_asg_ejec>, Correos=<valor_n_correos>"
df_simulaciones_info['Tratamiento'] = (
    'Ejecutivo=' + df_simulaciones_info['asg_ejec'].astype(str) +
    ', Correos=' + df_simulaciones_info['n_correos'].astype(int).astype(str)
)


In [23]:
# Extraer el mes y año de la columna 'fecha' y crear una nueva columna 'mes' en formato de periodo mensual
# Se convierte 'fecha' al formato datetime y luego se usa 'dt.to_period('M')' para obtener el mes/año.
df_simulaciones_info['mes'] = pd.to_datetime(df_simulaciones_info['fecha']).dt.to_period('M')

# Filtrar y mostrar las filas donde 'rut' es igual a 1
# Este filtro permite observar los registros específicos del cliente con 'rut' igual a 1, 
# lo cual es útil para verificar datos o analizar un cliente en particular.
df_simulaciones_info[df_simulaciones_info['rut'] == 1]


,Unnamed: 0_x,fecha,rut,n_correos,asg_ejec,Unnamed: 0_y,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Tratamiento,mes
0,0,2019-01-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-01
543651,0,2019-02-01,1,4,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=4",2019-02
1087302,0,2019-03-01,1,1,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=1",2019-03
1630953,0,2019-04-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-04
2174604,0,2019-05-01,1,0,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=0",2019-05
...,...,...,...,...,...,...,...,...,...,...,...
33162711,0,2024-02-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-02
33706362,0,2024-03-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-03
34250013,0,2024-04-01,1,2,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=2",2024-04
34793664,0,2024-05-01,1,2,1,7864785.0,267268.0,31.0,2.027869,"Ejecutivo=1, Correos=2",2024-05


In [24]:
# Crear una nueva columna 'simulo' para indicar si el cliente tiene un registro de simulación
# La columna 'Unnamed: 0_y' se utiliza para verificar si hay un valor no nulo, lo que implica que hay una simulación.
# 'notna()' devuelve True para valores no nulos y False para valores nulos; luego, 'astype(int)' convierte estos valores a 1 (True) o 0 (False).
df_simulaciones_info['simulo'] = df_simulaciones_info['Unnamed: 0_y'].notna().astype(int)

# Filtrar y mostrar las filas donde 'rut' es igual a 1
# Este filtro permite observar los registros específicos del cliente con 'rut' igual a 1, 
# útil para verificar si la columna 'simulo' refleja correctamente la presencia de simulaciones para este cliente.
df_simulaciones_info[df_simulaciones_info['rut'] == 1]


,Unnamed: 0_x,fecha,rut,n_correos,asg_ejec,Unnamed: 0_y,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Tratamiento,mes,simulo
0,0,2019-01-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-01,0
543651,0,2019-02-01,1,4,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=4",2019-02,0
1087302,0,2019-03-01,1,1,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=1",2019-03,0
1630953,0,2019-04-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-04,0
2174604,0,2019-05-01,1,0,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=0",2019-05,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33162711,0,2024-02-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-02,0
33706362,0,2024-03-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-03,0
34250013,0,2024-04-01,1,2,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=2",2024-04,0
34793664,0,2024-05-01,1,2,1,7864785.0,267268.0,31.0,2.027869,"Ejecutivo=1, Correos=2",2024-05,1


In [25]:
# Crear una copia del DataFrame 'df_estimar_elasticidad' con solo las columnas especificadas
# 'df1' contiene las columnas 'rut', 'categoria_clusterizacion_numerica', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
# Esta copia es útil para trabajar con los datos de elasticidad y clusterización sin modificar el DataFrame original.
df1 = df_estimar_elasticidad[['rut', 'categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']].copy()

# Crear una copia del DataFrame 'df_simulaciones_info' con solo las columnas especificadas
# 'df2' contiene las columnas 'rut', 'mes', 'Tratamiento' y 'simulo'.
# Esta copia es útil para trabajar con los datos de tratamiento y simulación en un conjunto de datos reducido.
df2 = df_simulaciones_info[['rut', 'mes', 'Tratamiento', 'simulo']].copy()


In [26]:
def function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad, df_simulaciones_info): #df1 es df_estimar_elasticidad y df2 es df_simulaciones_info
    # Paso 1: Preparación de datos y mapeo de clusters
    # Eliminar duplicados en 'df1' para tener un valor único de 'categoria_clusterizacion_numerica' por cada 'rut'.
    df_estimar_elasticidad_unique = df_estimar_elasticidad.drop_duplicates(subset='rut')

    # Crear un mapeo de 'rut' a 'categoria_clusterizacion_numerica' para asociar cada cliente a su cluster numérico.
    rut_cluster_map = df_estimar_elasticidad_unique.set_index('rut')['categoria_clusterizacion_numerica']

    # Mapear la categoría de cluster a cada 'rut' en 'df2' usando el mapeo creado
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['rut'].map(rut_cluster_map)

    # Eliminar filas donde 'categoria_clusterizacion_numerica' es nulo, es decir, aquellos 'rut' sin mapeo de cluster.
    df_simulaciones_info = df_simulaciones_info.dropna(subset=['categoria_clusterizacion_numerica'])

    # Conversión de tipos de datos
    # Convertir 'categoria_clusterizacion_numerica' a entero para garantizar un tipo de dato consistente.
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['categoria_clusterizacion_numerica'].astype(int)

    # Convertir 'simulo' a numérico, reemplazando valores nulos por 0 y asegurando que sea un tipo de dato entero.
    df_simulaciones_info['simulo'] = pd.to_numeric(df_simulaciones_info['simulo'], errors='coerce').fillna(0).astype(int)

    # Convertir 'Tratamiento' a tipo de categoría para optimizar espacio y realizar operaciones categóricas.
    df_simulaciones_info['Tratamiento'] = df_simulaciones_info['Tratamiento'].astype('category')

    # Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número total de registros en cada combinación.
    total_entries_per_cluster_treatment = df_simulaciones_info.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

    # Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
    # Filtrar filas donde 'simulo' es 1 (clientes que realizaron una simulación)
    df_simulations = df_simulaciones_info[df_simulaciones_info['simulo'] == 1]

    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número de registros favorables (simulaciones).
    favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

    # Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
    # Realizar un merge entre 'total_entries_per_cluster_treatment' y 'favorable_entries_per_cluster_treatment' en las columnas de cluster y tratamiento.
    df_probabilities = total_entries_per_cluster_treatment.merge(
        favorable_entries_per_cluster_treatment,
        on=['categoria_clusterizacion_numerica', 'Tratamiento'],
        how='left'
    )

    # Llenar valores nulos en 'caso_favorable' con 0, asegurando que solo las columnas numéricas estén afectadas.
    df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

    # Asegurar que 'caso_total' sea de tipo entero para evitar inconsistencias en los conteos.
    df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

    # Calcular la probabilidad de simulación como el cociente entre 'caso_favorable' y 'caso_total'.
    df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

    # Organizar las columnas del DataFrame resultante para facilitar su análisis.
    df_probabilities = df_probabilities[[
        'categoria_clusterizacion_numerica',
        'Tratamiento',
        'probabilidad_simular',
        'caso_favorable',
        'caso_total'
    ]]

    # Mostrar el DataFrame resultante con la probabilidad de simulación calculada para cada combinación de cluster y tratamiento.
    return df_probabilities


In [27]:
df_probabilities = function_estimar_respuesta_a_tratamiento(df1, df2)
df_probabilities

,categoria_clusterizacion_numerica,Tratamiento,probabilidad_simular,caso_favorable,caso_total
0,0,"Ejecutivo=0, Correos=0",0.238938,27,113
1,0,"Ejecutivo=0, Correos=1",0.201613,25,124
2,0,"Ejecutivo=0, Correos=2",0.222222,18,81
3,0,"Ejecutivo=0, Correos=3",0.247104,64,259
4,0,"Ejecutivo=0, Correos=4",0.250000,96,384
...,...,...,...,...,...
859,107,"Ejecutivo=0, Correos=3",0.152493,10569,69308
860,107,"Ejecutivo=0, Correos=4",0.152522,16988,111381
861,107,"Ejecutivo=1, Correos=0",0.341773,6007,17576
862,107,"Ejecutivo=1, Correos=1",0.349486,6664,19068


In [28]:
# Visualizar el DataFrame 'df_estimar_elasticidad' para revisar su contenido antes de realizar cálculos adicionales
df_estimar_elasticidad

# Calcular el valor promedio de 'Monto_Simulado' para cada 'categoria_clusterizacion_numerica'
# Usamos 'groupby' para agrupar por 'categoria_clusterizacion_numerica' y 'transform("mean")' para calcular el promedio.
# Luego, 'transform' asigna este valor promedio a cada fila dentro de su grupo, creando una columna 'Monto_Simulado_mean' con estos promedios.
df_estimar_elasticidad['Monto_Simulado_mean'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Monto_Simulado'].transform('mean')

# Calcular el valor promedio de 'Plazo_Simulado' para cada 'categoria_clusterizacion_numerica'
# Similar al cálculo anterior, 'groupby' agrupa los datos por 'categoria_clusterizacion_numerica', y 'transform("mean")' calcula el promedio.
# Se asigna el promedio resultante a cada fila dentro del grupo en la nueva columna 'Plazo_Simulado_mean'.
df_estimar_elasticidad['Plazo_Simulado_mean'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform('mean')
df_estimar_elasticidad['Plazo_Simulado_min'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform('min')
df_estimar_elasticidad['Plazo_Simulado_max'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform('max')
df_estimar_elasticidad['Plazo_Simulado_mode'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform(lambda x: x.mode().iloc[0])


In [29]:
# Seleccionar solo las columnas necesarias del DataFrame 'df_estimar_elasticidad' para reducir su tamaño
# 'df_estimar_elasticidad_small' contiene las columnas esenciales para el análisis:
# 'categoria_clusterizacion_numerica', 'rut', 'tasa_optima', 'probabilidad_aceptacion_optima', 'Probabilidad_No_Pago',
# 'Monto_Simulado_mean', y 'Plazo_Simulado_mean'.
df_estimar_elasticidad_small = df_estimar_elasticidad[['categoria_clusterizacion_numerica', 'rut', 'tasa_optima', 'probabilidad_aceptacion_optima', 'Probabilidad_No_Pago', 
                                                       'Monto_Simulado_mean',
                                                       'Plazo_Simulado_mean', 'Plazo_Simulado_min', 'Plazo_Simulado_max', 'Plazo_Simulado_mode']]

# Seleccionar solo las columnas necesarias del DataFrame 'df_probabilities' para reducir su tamaño
# 'df_probabilities_small' contiene las columnas 'categoria_clusterizacion_numerica', 'probabilidad_simular', y 'Tratamiento'.
df_probabilities_small = df_probabilities[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']]

# Realizar un merge entre 'df_estimar_elasticidad_small' y 'df_probabilities_small' usando 'categoria_clusterizacion_numerica' como clave
# Esta unión ('how="left"') mantiene todas las filas de 'df_estimar_elasticidad_small' y añade la información de 'df_probabilities_small'
# cuando hay coincidencias en 'categoria_clusterizacion_numerica'. El resultado se guarda en 'df_asignacion_de_tratamientos'.
df_asignacion_de_tratamientos = pd.merge(df_estimar_elasticidad_small, df_probabilities_small, on='categoria_clusterizacion_numerica', how='left')


In [30]:
# Crear un nombre de carpeta con una marca de tiempo actual
# 'strftime' genera la fecha y hora actual en el formato "YYYYMMDD_HHMMSS".
# Esto se usa para crear una carpeta única 'folder_name' donde se guardarán los archivos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"cluster_data_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si ya existe.

# Guardar información de los clusters
# Seleccionar las columnas relevantes sobre cada cluster desde 'df_estimar_elasticidad_small' y eliminar duplicados.
# El DataFrame 'df_cluster_info' contiene datos únicos de cada cluster como el monto y plazo medio simulado, la probabilidad de aceptación óptima y la tasa óptima.
df_cluster_info = df_estimar_elasticidad_small[['categoria_clusterizacion_numerica', 'probabilidad_aceptacion_optima', 'tasa_optima',
                                                'Monto_Simulado_mean',
                                                'Plazo_Simulado_mean', 'Plazo_Simulado_min', 'Plazo_Simulado_max', 'Plazo_Simulado_mode']].drop_duplicates()
df_cluster_info.to_csv(f"{folder_name}/cluster_info.csv", index=False)

# Guardar las probabilidades y tratamiento
# Seleccionar columnas relevantes de 'df_probabilities_small' para almacenar la probabilidad de simulación y tratamiento asignado para cada cluster.
# 'df_probabilities_treatment' contiene esta información única por cada combinación de cluster y tratamiento.
df_probabilities_treatment = df_probabilities_small[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']].drop_duplicates()
df_probabilities_treatment.to_csv(f"{folder_name}/probabilities_treatment.csv", index=False)

# Guardar información del RUT
# Seleccionar columnas clave sobre cada cliente ('rut') desde 'df_estimar_elasticidad_small' y eliminar duplicados.
# 'df_rut_info' contiene el 'rut', la categoría de cluster y la probabilidad de no pago para cada cliente, sin registros duplicados.
df_rut_info = df_estimar_elasticidad_small[['rut', 'categoria_clusterizacion_numerica', 'Probabilidad_No_Pago']].drop_duplicates()
df_rut_info.to_csv(f"{folder_name}/rut_info.csv", index=False)


# 6. Modelo de asignacion

## Modelo de asignacion que itera por cliente

In [31]:
# # Complete Optimized Code: Data Processing and Optimization

# import pandas as pd
# import os
# import numpy as np
# from gurobipy import Model, GRB, quicksum

# # -------------------------------
# # Data Processing and Preprocessing
# # -------------------------------

# # Define the base folder and mappings
# base_folder = "cluster_data_20241106_020021/"
# tratamiento_map = {
#     "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
#     "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
#     "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
#     "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
# }

# # File paths
# probabilities_file = os.path.join(base_folder, "probabilities_treatment.csv")
# rut_info_file = os.path.join(base_folder, "rut_info.csv")
# cluster_info_file = os.path.join(base_folder, "cluster_info.csv")

# # Parameters
# costosms = 100
# capacidad_ejecutivos = 205000

# # Step 1: Load and map 'tratamiento_id' in the probabilities data
# print("Loading and processing probabilities data...")
# df_probabilities = df_probabilities_treatment
# df_probabilities['tratamiento_id'] = df_probabilities['Tratamiento'].map(tratamiento_map)

# # Step 2: Create 'tratamientos' list and merge with rut_info
# print("Merging probabilities with rut_info...")
# df_probabilities['tratamientos'] = df_probabilities[['probabilidad_simular', 'tratamiento_id']].values.tolist()
# grouped_prob = df_probabilities.groupby('categoria_clusterizacion_numerica')['tratamientos'].apply(list).reset_index()

# df_rut_info = df_rut_info
# df_rut_info = df_rut_info.merge(grouped_prob, on='categoria_clusterizacion_numerica', how='left')

# # Step 3: Merge rut_info with cluster_info
# print("Merging rut_info with cluster_info...")
# df_cluster_info = df_cluster_info
# df_rut_info = df_rut_info.merge(df_cluster_info, on='categoria_clusterizacion_numerica', how='left')

# # Step 4: Calculate 'RC'
# print("Calculating RC...")
# df_rut_info['tasa_optima'] /= 100
# df_rut_info['RC'] = (
#     (df_rut_info['Plazo_Simulado_mean'] * df_rut_info['Monto_Simulado_mean'] * df_rut_info['tasa_optima'] *
#      ((1 + df_rut_info['tasa_optima']) ** df_rut_info['Plazo_Simulado_mean'])) /
#     (((1 + df_rut_info['tasa_optima']) ** df_rut_info['Plazo_Simulado_mean']) - 1)
# ) - df_rut_info['Monto_Simulado_mean']

# # -------------------------------
# # Data Preparation for Optimization
# # -------------------------------

# # Convert 'tratamientos' to numpy array to improve indexing performance
# profits = np.array([
#     [
#         (row['RC'] * (1 - row['Probabilidad_No_Pago']) * row['probabilidad_aceptacion_optima'] * row['tratamientos'][t][0]) - 
#         (row['tratamientos'][t][1] * costosms)
#         for t in range(8)
#     ]
#     for _, row in df_rut_info.iterrows()
# ])

# # Initialize model
# model = Model("Maximizar_Ganancias")
# model.ModelSense = GRB.MAXIMIZE

# # Create decision variables and set objective
# n_clients, n_treatments = profits.shape
# variables = {}

# for i in range(n_clients):
#     variables[i] = {}
#     for t in range(n_treatments):
#         if profits[i, t] > 0:
#             variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")

# model.setObjective(
#     quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
# )

# # Constraint: Each client receives exactly one treatment
# for i in variables:
#     model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")

# # Capacity constraint for executives
# model.addConstr(
#     quicksum(variables[i][t] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
#     name="CapacityConstraint"
# )

# # Cluster consistency
# clusters = df_rut_info.groupby("categoria_clusterizacion_numerica").indices
# for cluster_id, indices_cluster in clusters.items():
#     indices_list = list(indices_cluster)
#     leader_index = indices_list[0]
#     for t in variables[leader_index]:
#         leader_var = variables[leader_index][t]
#         for i in indices_list[1:]:
#             if t in variables[i]:
#                 model.addConstr(variables[i][t] == leader_var, name=f"ClusterConsistency_{cluster_id}_{t}")

In [32]:

# # Initialize model
# model = Model("Maximizar_Ganancias")
# model.ModelSense = GRB.MAXIMIZE

# # Create decision variables and set objective
# n_clients, n_treatments = profits.shape
# variables = {}

# for i in range(n_clients):
#     variables[i] = {}
#     for t in range(n_treatments):
#         if profits[i, t] > 0:
#             variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")

# model.setObjective(
#     quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
# )

# # Constraint: Each client receives exactly one treatment
# for i in variables:
#     model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")

# # Capacity constraint for executives
# model.addConstr(
#     quicksum(variables[i][t] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
#     name="CapacityConstraint"
# )

# # Cluster consistency
# clusters = df_rut_info.groupby("categoria_clusterizacion_numerica").indices
# for cluster_id, indices_cluster in clusters.items():
#     indices_list = list(indices_cluster)
#     leader_index = indices_list[0]
#     for t in variables[leader_index]:
#         leader_var = variables[leader_index][t]
#         for i in indices_list[1:]:
#             if t in variables[i]:
#                 model.addConstr(variables[i][t] == leader_var, name=f"ClusterConsistency_{cluster_id}_{t}")

# # Optimize the model
# model.optimize()

# # Check if the optimization was successful
# if model.Status == GRB.OPTIMAL:
#     # -------------------------------
#     # Extracting and Displaying Results
#     # -------------------------------

#     print("Extracting results...")

#     # Assign treatments per cluster based on the optimization results
#     resultados_por_cluster = {}
#     for cluster_id, indices_cluster in clusters.items():
#         leader_index = list(indices_cluster)[0]
#         for t in variables[leader_index]:
#             if variables[leader_index][t].X > 0.5:
#                 resultados_por_cluster[cluster_id] = t
#                 break

#     # Calculate total profits
#     ganancias_totales = model.ObjVal

#     # Display results
#     print("\nTratamientos asignados por cluster:")
#     for cluster_id, tratamiento in resultados_por_cluster.items():
#         print(f"Cluster {cluster_id}: Tratamiento {tratamiento + 1}")

#     print(f"\nGanancias totales: {ganancias_totales:.2f}")

#     # -------------------------------
#     # Calculating Executive Usage
#     # -------------------------------
    
#     # Count the number of executives used for treatments 5, 6, and 7
#     executives_used = sum(
#         1 for i in variables for t in variables[i] if t in [5, 6, 7] and variables[i][t].X > 0.5
#     )
#     executives_remaining = capacidad_ejecutivos - executives_used

#     # Display executive usage summary
#     print(f"\nExecutives used: {executives_used}")
#     print(f"Executives remaining: {executives_remaining}")

# else:
#     print("Optimization did not reach an optimal solution.")
# print("Optimization complete.")

In [33]:
# from collections import Counter

# for cluster_id, tratamiento in resultados_por_cluster.items():
#     # Count the number of times each treatment is assigned
#     treatment_counts = Counter(resultados_por_cluster.values())

# # Print the counts for each treatment
# for treatment, count in treatment_counts.items():
#     print(f"Treatment {treatment + 1}: {count} times")

## Modelo de asignacion que itera por cluster

In [34]:
# -------------------------------
# Procesamiento y preprocesamiento de datos
# -------------------------------

# Definir la carpeta base y el mapeo de tratamientos
base_folder = "cluster_data_20241106_020021/"  # Carpeta donde se encuentran los archivos de datos
tratamiento_map = {  # Mapeo de los tratamientos específicos a identificadores numéricos
    "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
}

# Definir las rutas de los archivos
probabilities_file = os.path.join(base_folder, "probabilities_treatment.csv")
rut_info_file = os.path.join(base_folder, "rut_info.csv")
cluster_info_file = os.path.join(base_folder, "cluster_info.csv")

# Parámetros
costosms = 100  # Costo de cada mensaje SMS
capacidad_ejecutivos = 205000  # Capacidad máxima en términos de tiempo de los ejecutivos

# Paso 1: Cargar y mapear 'tratamiento_id' en los datos de probabilidades
print("Loading and processing probabilities data...")
df_probabilities = df_probabilities_treatment
df_probabilities['tratamiento_id'] = df_probabilities['Tratamiento'].map(tratamiento_map)

# Paso 2: Crear lista de tratamientos y combinar con rut_info
print("Merging probabilities with rut_info...")
df_probabilities['tratamientos'] = df_probabilities[['probabilidad_simular', 'tratamiento_id']].values.tolist()
grouped_prob = df_probabilities.groupby('categoria_clusterizacion_numerica')['tratamientos'].apply(list).reset_index()

df_rut_info = df_rut_info
df_rut_info = df_rut_info.merge(grouped_prob, on='categoria_clusterizacion_numerica', how='left')

# Paso 3: Combinar rut_info con cluster_info
print("Merging rut_info with cluster_info...")
df_cluster_info = df_cluster_info
df_rut_info = df_rut_info.merge(df_cluster_info, on='categoria_clusterizacion_numerica', how='left')

# Paso 3.5: Agrupar información por cluster en 'rut_info'
# Agrupar por 'categoria_clusterizacion_numerica' y agregar según lo especificado
df_grouped = df_rut_info.groupby('categoria_clusterizacion_numerica').agg({
    'Probabilidad_No_Pago': 'mean',  # Promedio de probabilidad de no pago
    'tratamientos': lambda x: list(x),  # Lista de opciones de tratamiento únicas en cada cluster
    'Monto_Simulado_mean': 'mean',
    'Plazo_Simulado_mean': 'mean',
    'probabilidad_aceptacion_optima': 'mean',
    'tasa_optima': 'mean',
    'rut': 'count'  # Conteo del número de clientes ('rut') en cada cluster
}).rename(columns={'rut': 'n_clientes'}).reset_index()

# Paso 4: Calcular 'RC' (Revenue calculado)
print("Calculating RC...")
df_grouped['tasa_optima'] /= 100  # Convertir tasa óptima a decimal
df_grouped['RC'] = (
    (df_grouped['Plazo_Simulado_mean'] * df_grouped['Monto_Simulado_mean'] * df_grouped['tasa_optima'] *
     ((1 + df_grouped['tasa_optima']) ** df_grouped['Plazo_Simulado_mean'])) /
    (((1 + df_grouped['tasa_optima']) ** df_grouped['Plazo_Simulado_mean']) - 1)
) - df_grouped['Monto_Simulado_mean']

# -------------------------------
# Preparación de datos para optimización
# -------------------------------

# Convertir 'tratamientos' a un arreglo de numpy para mejorar la indexación
# Desarrollar y preparar 'tratamientos' para indexación adecuada
profits = np.array([
    [
        row['n_clientes'] * (row['RC'] * (1 - row['Probabilidad_No_Pago']) * row['probabilidad_aceptacion_optima'] * row['tratamientos'][0][t][0]) - 
        (row['tratamientos'][0][t][1] * costosms)
        for t in range(8)
    ]
    for _, row in df_grouped.iterrows()
])

# Inicializar el modelo de optimización
model = Model("Maximizar_Ganancias")
model.ModelSense = GRB.MAXIMIZE

# Crear variables de decisión y definir el objetivo
n_clients, n_treatments = profits.shape
variables = {}

for i in range(n_clients):
    variables[i] = {}
    for t in range(n_treatments):
        if profits[i, t] > 0:
            variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")

model.setObjective(
    quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
)

# Restricción: Cada cliente recibe exactamente un tratamiento
for i in variables:
    model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")

# Restricción de capacidad para ejecutivos
model.addConstr(
    quicksum(variables[i][t] * df_grouped.loc[i, 'n_clientes'] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
    name="CapacityConstraint"
)

# Consistencia de cluster: los clientes dentro del mismo cluster deben recibir el mismo tratamiento
clusters = df_grouped.groupby("categoria_clusterizacion_numerica").indices
for cluster_id, indices_cluster in clusters.items():
    indices_list = list(indices_cluster)
    leader_index = indices_list[0]
    for t in variables[leader_index]:
        leader_var = variables[leader_index][t]
        for i in indices_list[1:]:
            if t in variables[i]:
                model.addConstr(variables[i][t] == leader_var, name=f"ClusterConsistency_{cluster_id}_{t}")

# Optimizar el modelo
model.optimize()

# Verificar si la optimización fue exitosa
if model.Status == GRB.OPTIMAL:
    # -------------------------------
    # Extracción y visualización de resultados
    # -------------------------------

    print("Extracting results...")

    # Asignar tratamientos por cluster basado en los resultados de la optimización
    resultados_por_cluster = {}
    for cluster_id, indices_cluster in clusters.items():
        leader_index = list(indices_cluster)[0]
        for t in variables[leader_index]:
            if variables[leader_index][t].X > 0.5:
                resultados_por_cluster[cluster_id] = t + 1
                break

    # Calcular las ganancias totales
    ganancias_totales = model.ObjVal

    # Mostrar resultados
    print("\nTratamientos asignados por cluster:")
    for cluster_id, tratamiento in resultados_por_cluster.items():
        print(f"Cluster {cluster_id}: Tratamiento {tratamiento}")

    print(f"\nGanancias totales: {ganancias_totales:.2f}")

    # Calcular el número de ejecutivos usados y restantes
    executives_used = sum(
        df_grouped.loc[i, 'n_clientes'] for i in variables for t in variables[i]
        if t in [5, 6, 7] and variables[i][t].X > 0.5
    )
    executives_remaining = capacidad_ejecutivos - executives_used

    # Mostrar resumen de uso de ejecutivos
    print(f"\nExecutives used: {executives_used}")
    print(f"Executives remaining: {executives_remaining}")
else:
    print("Optimization did not reach an optimal solution.")
print("Optimization complete.")


Loading and processing probabilities data...
Merging probabilities with rut_info...
Merging rut_info with cluster_info...
Calculating RC...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-06
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 109 rows, 864 columns and 1188 nonzeros
Model fingerprint: 0xb58db2b5
Variable types: 0 continuous, 864 integer (864 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [3e+04, 1e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 5.130598e+10
Presolve removed 108 rows and 756 columns
Presolve time

### Resultados

In [35]:
# Contar cuántas veces se asigna cada tratamiento en los resultados por cluster
# Se utiliza un diccionario 'Counter' para contar las ocurrencias de cada tratamiento asignado en 'resultados_por_cluster'
for cluster_id, tratamiento in resultados_por_cluster.items():
    treatment_counts = Counter(resultados_por_cluster.values())

# Imprimir el conteo de asignaciones para cada tratamiento
# Se recorre 'treatment_counts' para mostrar cuántas veces se asignó cada tratamiento.
# 'treatment + 1' se utiliza para mostrar el número de tratamiento en base 1, haciendo el resultado más legible.
for treatment, count in treatment_counts.items():
    print(f"Treatment {treatment}: {count} times")


Treatment 8: 38 times
Treatment 1: 2 times
Treatment 7: 10 times
Treatment 6: 2 times
Treatment 4: 10 times
Treatment 5: 41 times
Treatment 3: 5 times


In [36]:
# Convertir el diccionario 'resultados_por_cluster' a un DataFrame
# El diccionario 'resultados_por_cluster' contiene el ID del cluster y el tratamiento asignado a cada uno.
# Se convierte a un DataFrame donde la primera columna es 'cluster' y la segunda 'assigned_treatment'.
df_resultados_por_cluster = pd.DataFrame(list(resultados_por_cluster.items()), columns=["cluster", "assigned_treatment"])

# Mostrar el DataFrame resultante al usuario
df_resultados_por_cluster


,cluster,assigned_treatment
0,0,8
1,1,1
2,2,7
3,3,6
4,4,4
...,...,...
103,103,5
104,104,5
105,105,7
106,106,5


In [37]:
# Primer merge con 'df_rut_info'
# Realizar una unión ('merge') entre 'df_resultados_por_cluster' y 'df_rut_info' usando 'cluster' en el primer DataFrame
# y 'categoria_clusterizacion_numerica' en el segundo como claves de unión.
# Esta unión permite agregar información de clientes a cada cluster con su tratamiento asignado.
df_assigned = pd.merge(df_resultados_por_cluster, df_rut_info, left_on='cluster', right_on='categoria_clusterizacion_numerica', how='left')

# Segundo merge con 'df_grouped' para agregar la columna 'RC'
# Realizar una unión entre 'df_assigned' y 'df_grouped' para incorporar la columna 'RC' (Revenue Calculado)
# Usamos 'categoria_clusterizacion_numerica' como clave de unión para añadir la información de revenue calculado a cada cluster.
df_assigned = pd.merge(df_assigned, df_grouped[['categoria_clusterizacion_numerica', 'RC']], on='categoria_clusterizacion_numerica', how='left')


In [38]:
# Calcular la probabilidad de simulación para el tratamiento asignado en cada fila de 'df_assigned'
# Se usa 'apply' con una función lambda para extraer la probabilidad de simulación correspondiente al tratamiento asignado.
# 'row['tratamientos']' es una lista de opciones de tratamiento y 'row['assigned_treatment'] - 1' indica la posición del tratamiento.
# Primero, se verifica que 'tratamientos' sea una lista y que el índice calculado esté dentro del rango.
# Si estas condiciones se cumplen, se extrae la probabilidad de simulación; en caso contrario, se asigna None.
df_assigned['probabilidad_de_simular'] = df_assigned.apply(
    lambda row: row['tratamientos'][row['assigned_treatment'] - 1][0] 
                if isinstance(row['tratamientos'], list) and (0 <= row['assigned_treatment'] - 1 < len(row['tratamientos'])) 
                else None,
    axis=1
)


In [39]:
import os
from datetime import datetime

# Crear una carpeta con un nombre basado en la fecha y hora actual
# 'strftime' genera un timestamp en el formato "YYYYMMDD_HHMMSS" para asegurar nombres de carpeta únicos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"assigned_treatments/assignation_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si la carpeta ya existe.

# Definir la ruta del archivo CSV dentro de la nueva carpeta
output_path = os.path.join(folder_name, 'assigned_treatments.csv')

# Guardar el DataFrame 'df_assigned' con las columnas seleccionadas en un archivo CSV
# Se incluyen las columnas clave: 'rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment',
# 'probabilidad_de_simular', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
df_assigned[['rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment', 'probabilidad_de_simular', 'tasa_optima', 'probabilidad_aceptacion_optima']].to_csv(output_path, index=False)

# Imprimir mensaje de confirmación con la ubicación del archivo CSV guardado
print(f"CSV file saved in folder: {output_path}")


CSV file saved in folder: assigned_treatments/assignation_20241127_184711\assigned_treatments.csv


# MODELO INTUITIVO 1

- Enviar el maximo de correos a todos (de forma aleatoria a quienes asignarles ejecutivos) 
- Precios aleatorios también

In [40]:
df_rut_info_int1 = df_rut_info.copy()
df_probabilities_treatment_int1 = df_probabilities_treatment.copy()
df_cluster_info_int1 = df_cluster_info.copy()
df_simulaciones_info_int_1 = df_simulaciones_info.copy()

KeyboardInterrupt: 

In [ ]:
df_estimar_elasticidad_int1 = df_estimar_elasticidad.copy()

In [ ]:
df_rut_info_int1['categoria_clusterizacion_numerica'] = 0

## Precios intuitivo 1

In [ ]:
df_estimar_elasticidad_int1['categoria_clusterizacion_numerica'] = 0

In [ ]:
def function_estimar_elasticidad_int1(df):
    """
    Assigns random prices to clients in a single cluster and estimates the probability 
    of acceptance for each price using logistic regression.

    Parameters:
    - df (pd.DataFrame): DataFrame containing client data with at least 'venta', 
                         'Tasa_Simulado', and 'rut' columns.

    Returns:
    - pd.DataFrame: DataFrame with 'rut', 'price_set', and 'acceptance_probability'.
    """
    
    # Check if the DataFrame is empty
    if df.empty:
        print("The input DataFrame is empty.")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Clean the data: Remove rows with NaN or infinite values in relevant columns
    df_clean = df.replace([np.inf, -np.inf], np.nan).dropna(
        subset=['venta', 'Tasa_Simulado']
    )
    
    # Ensure there are enough data points to fit the model
    if df_clean.shape[0] < 10:
        print("Not enough data to fit the logistic regression model.")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Define the dependent and independent variables
    y = df_clean['venta']
    X = df_clean[['Tasa_Simulado']]
    
    # Add a constant term for the intercept
    X = sm.add_constant(X)
    
    # Fit the logistic regression model
    try:
        logit_model = sm.Logit(y, X)
        result = logit_model.fit(disp=0)
    except Exception as e:
        print(f"Logistic regression failed to converge: {e}")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Determine the range of prices from the data
    tasa_min = df_clean['Tasa_Simulado'].min()
    tasa_max = df_clean['Tasa_Simulado'].max()

    df_clean = df_clean.drop_duplicates(subset='rut')
    
    # Assign a random price to each client within the observed range
    np.random.seed(42)  # For reproducibility; remove or change seed as needed
    df_clean['tasa_optima'] = np.random.uniform(low=tasa_min, high=tasa_max, size=df_clean.shape[0])

    
    # Prepare the data for prediction by adding a constant term
    X_new = sm.add_constant(df_clean['tasa_optima'])
    
    # Predict the acceptance probability using the logistic model
    df_clean['probabilidad_aceptacion_optima'] = result.predict(X_new)
    
    # Ensure probabilities are within [0, 1]
    df_clean['probabilidad_aceptacion_optima'] = df_clean['probabilidad_aceptacion_optima'].clip(0, 1)
    
    # Select the required columns to return
    result_df = df_clean[['rut', 'tasa_optima', 'probabilidad_aceptacion_optima']].copy()
    
    return result_df

In [ ]:
prices = function_estimar_elasticidad_int1(df_estimar_elasticidad_int1)

In [ ]:
prices

,rut,tasa_optima,probabilidad_aceptacion_optima
0,1,1.561810,0.383343
15,2,2.426071,0.117775
25,3,2.097991,0.193142
45,4,1.897988,0.254692
70,5,1.234028,0.526982
...,...,...,...
8517002,543647,2.137431,0.182437
8517010,543648,1.223011,0.531868
8517028,543649,1.562318,0.383129
8517042,543650,1.341532,0.479184


## Tratamientos intuitivo 1

### Estimacion de respuesta

In [ ]:
df_probabilities_treatment_int1 = df_probabilities_treatment_int1[(df_probabilities_treatment_int1['Tratamiento'] == '	Ejecutivo=0, Correos=4')|(df_probabilities_treatment_int1['Tratamiento'] == '	Ejecutivo=1, Correos=2')]

In [ ]:
tratamiento_map = {  # Mapeo de los tratamientos específicos a identificadores numéricos
    "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
}
df_probabilities_int1 = df_probabilities_treatment_int1
df_probabilities_int1['tratamiento_id'] = df_probabilities_int1['Tratamiento'].map(tratamiento_map)

In [ ]:
def function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad, df_simulaciones_info): #df1 es df_estimar_elasticidad y df2 es df_simulaciones_info
    # Paso 1: Preparación de datos y mapeo de clusters
    # Eliminar duplicados en 'df1' para tener un valor único de 'categoria_clusterizacion_numerica' por cada 'rut'.
    df_estimar_elasticidad_unique = df_estimar_elasticidad.drop_duplicates(subset='rut')

    # Crear un mapeo de 'rut' a 'categoria_clusterizacion_numerica' para asociar cada cliente a su cluster numérico.
    rut_cluster_map = df_estimar_elasticidad_unique.set_index('rut')['categoria_clusterizacion_numerica']

    # Mapear la categoría de cluster a cada 'rut' en 'df2' usando el mapeo creado
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['rut'].map(rut_cluster_map)

    # Eliminar filas donde 'categoria_clusterizacion_numerica' es nulo, es decir, aquellos 'rut' sin mapeo de cluster.
    df_simulaciones_info = df_simulaciones_info.dropna(subset=['categoria_clusterizacion_numerica'])

    # Conversión de tipos de datos
    # Convertir 'categoria_clusterizacion_numerica' a entero para garantizar un tipo de dato consistente.
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['categoria_clusterizacion_numerica'].astype(int)

    # Convertir 'simulo' a numérico, reemplazando valores nulos por 0 y asegurando que sea un tipo de dato entero.
    df_simulaciones_info['simulo'] = pd.to_numeric(df_simulaciones_info['simulo'], errors='coerce').fillna(0).astype(int)

    # Convertir 'Tratamiento' a tipo de categoría para optimizar espacio y realizar operaciones categóricas.
    df_simulaciones_info['Tratamiento'] = df_simulaciones_info['Tratamiento'].astype('category')

    # Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número total de registros en cada combinación.
    total_entries_per_cluster_treatment = df_simulaciones_info.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

    # Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
    # Filtrar filas donde 'simulo' es 1 (clientes que realizaron una simulación)
    df_simulations = df_simulaciones_info[df_simulaciones_info['simulo'] == 1]

    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número de registros favorables (simulaciones).
    favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

    # Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
    # Realizar un merge entre 'total_entries_per_cluster_treatment' y 'favorable_entries_per_cluster_treatment' en las columnas de cluster y tratamiento.
    df_probabilities = total_entries_per_cluster_treatment.merge(
        favorable_entries_per_cluster_treatment,
        on=['categoria_clusterizacion_numerica', 'Tratamiento'],
        how='left'
    )

    # Llenar valores nulos en 'caso_favorable' con 0, asegurando que solo las columnas numéricas estén afectadas.
    df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

    # Asegurar que 'caso_total' sea de tipo entero para evitar inconsistencias en los conteos.
    df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

    # Calcular la probabilidad de simulación como el cociente entre 'caso_favorable' y 'caso_total'.
    df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

    # Organizar las columnas del DataFrame resultante para facilitar su análisis.
    df_probabilities = df_probabilities[[
        'categoria_clusterizacion_numerica',
        'Tratamiento',
        'probabilidad_simular',
        'caso_favorable',
        'caso_total'
    ]]

    # Mostrar el DataFrame resultante con la probabilidad de simulación calculada para cada combinación de cluster y tratamiento.
    return df_probabilities


In [ ]:
df_probabilities = function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad_int1, df_simulaciones_info_int_1)
df_probabilities

,categoria_clusterizacion_numerica,Tratamiento,probabilidad_simular,caso_favorable,caso_total
0,0,"Ejecutivo=0, Correos=0",0.142747,393437,2756192
1,0,"Ejecutivo=0, Correos=1",0.154606,461210,2983137
2,0,"Ejecutivo=0, Correos=2",0.166960,383438,2296582
3,0,"Ejecutivo=0, Correos=3",0.172610,990544,5738616
4,0,"Ejecutivo=0, Correos=4",0.175640,1612628,9181465
5,0,"Ejecutivo=1, Correos=0",0.342493,531665,1552340
6,0,"Ejecutivo=1, Correos=1",0.353799,595294,1682576
7,0,"Ejecutivo=1, Correos=2",0.366161,3548122,9690058


### Asignación

In [ ]:
# Assign default treatment value of 5 to all rows
df_rut_info_int1['assigned_treatment'] = 5

# Randomly select 205,000 unique indices without replacement
random_indices = df_rut_info_int1.sample(n=205000, random_state=42).index

# Assign treatment value of 8 to the selected random rows
df_rut_info_int1.loc[random_indices, 'assigned_treatment'] = 8

# Retrieve the required probabilities from df_probabilities
prob_for_5 = df_probabilities.loc[4, 'probabilidad_simular']
prob_for_8 = df_probabilities.loc[7, 'probabilidad_simular']

# Add the new column based on 'assigned_treatment'
df_rut_info_int1['probabilidad_de_simular'] = df_rut_info_int1['assigned_treatment'].map({
    5: prob_for_5,
    8: prob_for_8
})


In [ ]:
prices

,rut,tasa_optima,probabilidad_aceptacion_optima
0,1,1.561810,0.383343
15,2,2.426071,0.117775
25,3,2.097991,0.193142
45,4,1.897988,0.254692
70,5,1.234028,0.526982
...,...,...,...
8517002,543647,2.137431,0.182437
8517010,543648,1.223011,0.531868
8517028,543649,1.562318,0.383129
8517042,543650,1.341532,0.479184


In [ ]:
assigned_treatments_int1 = pd.merge(df_rut_info_int1, prices, on='rut', how='left')

In [ ]:
df_cluster_info_int1_csv = df_estimar_elasticidad_int1.groupby('categoria_clusterizacion_numerica').agg({
'Monto_Simulado_mean': 'mean',
'Plazo_Simulado_mean': 'mean',
'Plazo_Simulado_min': 'min',
'Plazo_Simulado_max': 'max',
'Plazo_Simulado_mode': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan})

In [ ]:
assigned_treatments_int1 = assigned_treatments_int1.rename(columns={'categoria_clusterizacion_numerica': 'cluster'})

In [ ]:
import os
from datetime import datetime

# Crear una carpeta con un nombre basado en la fecha y hora actual
# 'strftime' genera un timestamp en el formato "YYYYMMDD_HHMMSS" para asegurar nombres de carpeta únicos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"assigned_treatments_intuitivo_1/assignation_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si la carpeta ya existe.

# Definir la ruta del archivo CSV dentro de la nueva carpeta
output_path = os.path.join(folder_name, 'assigned_treatments_intuitivo1.csv')
output_path_cluster = os.path.join(folder_name, 'cluster_info_intuitivo1.csv')

# Guardar el DataFrame 'df_assigned' con las columnas seleccionadas en un archivo CSV
# Se incluyen las columnas clave: 'rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment',
# 'probabilidad_de_simular', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
assigned_treatments_int1[['rut', 'cluster', 'Probabilidad_No_Pago', 'assigned_treatment', 'probabilidad_de_simular', 'tasa_optima',  'probabilidad_aceptacion_optima']].to_csv(output_path, index=False)
df_cluster_info_int1_csv.to_csv(output_path_cluster, index=True)
# Imprimir mensaje de confirmación con la ubicación del archivo CSV guardado
print(f"CSV file saved in folder: {output_path}")

CSV file saved in folder: assigned_treatments_intuitivo_1/assignation_20241116_175536\assigned_treatments_intuitivo1.csv


# MODELO INTUITIVO 2

- Enviar el mínimo de correos a todos (de forma aleatoria a quienes asignarles ejecutivos)
- Precios aleatorios también

In [ ]:
df_rut_info_int2 = df_rut_info.copy()
df_probabilities_treatment_int2 = df_probabilities_treatment.copy()
df_cluster_info_int2 = df_cluster_info.copy()
df_simulaciones_info_int_2 = df_simulaciones_info.copy()

In [ ]:
df_estimar_elasticidad_int2 = df_estimar_elasticidad.copy()

In [ ]:
df_rut_info_int2['categoria_clusterizacion_numerica'] = 0

## Precios intuitivo 2

In [ ]:
df_estimar_elasticidad_int2['categoria_clusterizacion_numerica'] = 0

In [ ]:
def function_estimar_elasticidad_int2(df):
    """
    Assigns random prices to clients in a single cluster and estimates the probability 
    of acceptance for each price using logistic regression.

    Parameters:
    - df (pd.DataFrame): DataFrame containing client data with at least 'venta', 
                         'Tasa_Simulado', and 'rut' columns.

    Returns:
    - pd.DataFrame: DataFrame with 'rut', 'price_set', and 'acceptance_probability'.
    """
    
    # Check if the DataFrame is empty
    if df.empty:
        print("The input DataFrame is empty.")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Clean the data: Remove rows with NaN or infinite values in relevant columns
    df_clean = df.replace([np.inf, -np.inf], np.nan).dropna(
        subset=['venta', 'Tasa_Simulado']
    )
    
    # Ensure there are enough data points to fit the model
    if df_clean.shape[0] < 10:
        print("Not enough data to fit the logistic regression model.")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Define the dependent and independent variables
    y = df_clean['venta']
    X = df_clean[['Tasa_Simulado']]
    
    # Add a constant term for the intercept
    X = sm.add_constant(X)
    
    # Fit the logistic regression model
    try:
        logit_model = sm.Logit(y, X)
        result = logit_model.fit(disp=0)
    except Exception as e:
        print(f"Logistic regression failed to converge: {e}")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Determine the range of prices from the data
    tasa_min = df_clean['Tasa_Simulado'].min()
    tasa_max = df_clean['Tasa_Simulado'].max()

    df_clean = df_clean.drop_duplicates(subset='rut')
    
    # Assign a random price to each client within the observed range
    np.random.seed(42)  # For reproducibility; remove or change seed as needed
    df_clean['tasa_optima'] = np.random.uniform(low=tasa_min, high=tasa_max, size=df_clean.shape[0])

    
    # Prepare the data for prediction by adding a constant term
    X_new = sm.add_constant(df_clean['tasa_optima'])
    
    # Predict the acceptance probability using the logistic model
    df_clean['probabilidad_aceptacion_optima'] = result.predict(X_new)
    
    # Ensure probabilities are within [0, 1]
    df_clean['probabilidad_aceptacion_optima'] = df_clean['probabilidad_aceptacion_optima'].clip(0, 1)
    
    # Select the required columns to return
    result_df = df_clean[['rut', 'tasa_optima', 'probabilidad_aceptacion_optima']].copy()
    
    return result_df

In [ ]:
prices = function_estimar_elasticidad_int2(df_estimar_elasticidad_int2)

## Tratamientos intuitivo 2

### Estimacion de respuesta

In [ ]:
df_probabilities_treatment_int2 = df_probabilities_treatment_int2[(df_probabilities_treatment_int2['Tratamiento'] == '	Ejecutivo=0, Correos=0')|(df_probabilities_treatment_int2['Tratamiento'] == '	Ejecutivo=1, Correos=0')]

In [ ]:
tratamiento_map = {  # Mapeo de los tratamientos específicos a identificadores numéricos
    "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
}
df_probabilities_int2 = df_probabilities_treatment_int2
df_probabilities_int2['tratamiento_id'] = df_probabilities_int2['Tratamiento'].map(tratamiento_map)

In [ ]:
def function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad, df_simulaciones_info): #df1 es df_estimar_elasticidad y df2 es df_simulaciones_info
    # Paso 1: Preparación de datos y mapeo de clusters
    # Eliminar duplicados en 'df1' para tener un valor único de 'categoria_clusterizacion_numerica' por cada 'rut'.
    df_estimar_elasticidad_unique = df_estimar_elasticidad.drop_duplicates(subset='rut')

    # Crear un mapeo de 'rut' a 'categoria_clusterizacion_numerica' para asociar cada cliente a su cluster numérico.
    rut_cluster_map = df_estimar_elasticidad_unique.set_index('rut')['categoria_clusterizacion_numerica']

    # Mapear la categoría de cluster a cada 'rut' en 'df2' usando el mapeo creado
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['rut'].map(rut_cluster_map)

    # Eliminar filas donde 'categoria_clusterizacion_numerica' es nulo, es decir, aquellos 'rut' sin mapeo de cluster.
    df_simulaciones_info = df_simulaciones_info.dropna(subset=['categoria_clusterizacion_numerica'])

    # Conversión de tipos de datos
    # Convertir 'categoria_clusterizacion_numerica' a entero para garantizar un tipo de dato consistente.
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['categoria_clusterizacion_numerica'].astype(int)

    # Convertir 'simulo' a numérico, reemplazando valores nulos por 0 y asegurando que sea un tipo de dato entero.
    df_simulaciones_info['simulo'] = pd.to_numeric(df_simulaciones_info['simulo'], errors='coerce').fillna(0).astype(int)

    # Convertir 'Tratamiento' a tipo de categoría para optimizar espacio y realizar operaciones categóricas.
    df_simulaciones_info['Tratamiento'] = df_simulaciones_info['Tratamiento'].astype('category')

    # Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número total de registros en cada combinación.
    total_entries_per_cluster_treatment = df_simulaciones_info.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

    # Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
    # Filtrar filas donde 'simulo' es 1 (clientes que realizaron una simulación)
    df_simulations = df_simulaciones_info[df_simulaciones_info['simulo'] == 1]

    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número de registros favorables (simulaciones).
    favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

    # Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
    # Realizar un merge entre 'total_entries_per_cluster_treatment' y 'favorable_entries_per_cluster_treatment' en las columnas de cluster y tratamiento.
    df_probabilities = total_entries_per_cluster_treatment.merge(
        favorable_entries_per_cluster_treatment,
        on=['categoria_clusterizacion_numerica', 'Tratamiento'],
        how='left'
    )

    # Llenar valores nulos en 'caso_favorable' con 0, asegurando que solo las columnas numéricas estén afectadas.
    df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

    # Asegurar que 'caso_total' sea de tipo entero para evitar inconsistencias en los conteos.
    df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

    # Calcular la probabilidad de simulación como el cociente entre 'caso_favorable' y 'caso_total'.
    df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

    # Organizar las columnas del DataFrame resultante para facilitar su análisis.
    df_probabilities = df_probabilities[[
        'categoria_clusterizacion_numerica',
        'Tratamiento',
        'probabilidad_simular',
        'caso_favorable',
        'caso_total'
    ]]

    # Mostrar el DataFrame resultante con la probabilidad de simulación calculada para cada combinación de cluster y tratamiento.
    return df_probabilities


In [ ]:
df_probabilities = function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad_int2, df_simulaciones_info_int_2)

### Asignación

In [ ]:
# Assign default treatment value of 1 to all rows
df_rut_info_int2['assigned_treatment'] = 1

# Randomly select 205,000 unique indices without replacement
random_indices = df_rut_info_int2.sample(n=205000, random_state=42).index

# Assign treatment value of 8 to the selected random rows
df_rut_info_int2.loc[random_indices, 'assigned_treatment'] = 6

# Retrieve the required probabilities from df_probabilities
prob_for_1 = df_probabilities.loc[0, 'probabilidad_simular']
prob_for_6 = df_probabilities.loc[5, 'probabilidad_simular']

# Add the new column based on 'assigned_treatment'
df_rut_info_int2['probabilidad_de_simular'] = df_rut_info_int2['assigned_treatment'].map({
    1: prob_for_1,
    6: prob_for_6
})


In [ ]:
assigned_treatments_int2 = pd.merge(df_rut_info_int2, prices, on='rut', how='left')

In [ ]:
assigned_treatments_int2.rename(columns={'categoria_clusterizacion_numerica': 'cluster'}, inplace=True)

In [ ]:
assigned_treatments_int2.rename(columns={'probabilidad_aceptacion_optima_x': 'probabilidad_aceptacion_optima', 'tasa_optima_x': 'tasa_optima'}, inplace=True)

In [ ]:
assigned_treatments_int2

,rut,cluster,Probabilidad_No_Pago,tratamientos,probabilidad_aceptacion_optima,tasa_optima,Monto_Simulado_mean,Plazo_Simulado_mean,Plazo_Simulado_min,Plazo_Simulado_max,Plazo_Simulado_mode,assigned_treatment,probabilidad_de_simular,tasa_optima_y,probabilidad_aceptacion_optima_y
0,1,0,0.028445,"[[0.20384870204179967, 1.0], [0.20784574100201...",0.499757,1.261287,2.672066e+06,27.465434,8.0,48.0,30.0,1,0.142747,1.561810,0.383343
1,2,0,0.014320,"[[0.08183039140445127, 1.0], [0.08207471588380...",0.552339,1.219217,7.124037e+05,27.651125,9.0,50.0,30.0,1,0.142747,2.426071,0.117775
2,3,0,0.002156,"[[0.20574240514050998, 1.0], [0.21240180638305...",0.624972,1.075107,2.459338e+07,27.514339,9.0,49.0,30.0,6,0.342493,2.097991,0.193142
3,4,0,0.034418,"[[0.20302978806049865, 1.0], [0.21238682120531...",0.473732,1.373885,7.612776e+05,27.501192,8.0,48.0,31.0,1,0.142747,1.897988,0.254692
4,5,0,0.014978,"[[0.14013414937103366, 1.0], [0.16421079183570...",0.499332,1.265790,4.997373e+05,27.442591,8.0,49.0,30.0,1,0.142747,1.234028,0.526982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543646,543647,0,0.037291,"[[0.07533101699690112, 1.0], [0.09850054429890...",0.496040,1.328843,5.326978e+05,27.408767,8.0,52.0,30.0,1,0.142747,2.137431,0.182437
543647,543648,0,0.035877,"[[0.20384870204179967, 1.0], [0.20784574100201...",0.499757,1.261287,2.672066e+06,27.465434,8.0,48.0,30.0,1,0.142747,1.223011,0.531868
543648,543649,0,0.023306,"[[0.0789925899981393, 1.0], [0.081787777439951...",0.497598,1.307808,4.327878e+06,27.510854,8.0,49.0,30.0,1,0.142747,1.562318,0.383129
543649,543650,0,0.015121,"[[0.2075966163542876, 1.0], [0.208343603648015...",0.568717,1.189190,4.125397e+06,27.479876,9.0,49.0,30.0,6,0.342493,1.341532,0.479184


In [ ]:
df_cluster_info_int2_csv = df_estimar_elasticidad_int2.groupby('categoria_clusterizacion_numerica').agg({
'Monto_Simulado_mean': 'mean',
'Plazo_Simulado_mean': 'mean',
'Plazo_Simulado_min': 'min',
'Plazo_Simulado_max': 'max',
'Plazo_Simulado_mode': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan})

In [ ]:
df_cluster_info_int2_csv

,Monto_Simulado_mean,Plazo_Simulado_mean,Plazo_Simulado_min,Plazo_Simulado_max,Plazo_Simulado_mode
categoria_clusterizacion_numerica,,,,,
0,6.964109e+06,27.502137,7.0,52.0,30.0


In [ ]:
import os
from datetime import datetime

# Crear una carpeta con un nombre basado en la fecha y hora actual
# 'strftime' genera un timestamp en el formato "YYYYMMDD_HHMMSS" para asegurar nombres de carpeta únicos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"assigned_treatments_intuitivo_2/assignation_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si la carpeta ya existe.

# Definir la ruta del archivo CSV dentro de la nueva carpeta
output_path = os.path.join(folder_name, 'assigned_treatments_intuitivo2.csv')
output_path_cluster = os.path.join(folder_name, 'cluster_info_intuitivo2.csv')

# Guardar el DataFrame 'df_assigned' con las columnas seleccionadas en un archivo CSV
# Se incluyen las columnas clave: 'rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment',
# 'probabilidad_de_simular', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
assigned_treatments_int2[['rut', 'cluster', 'Probabilidad_No_Pago', 'assigned_treatment', 'probabilidad_de_simular', 'tasa_optima',  'probabilidad_aceptacion_optima']].to_csv(output_path, index=False)
df_cluster_info_int2_csv.to_csv(output_path_cluster, index=True)
# Imprimir mensaje de confirmación con la ubicación del archivo CSV guardado
print(f"CSV file saved in folder: {output_path}")

CSV file saved in folder: assigned_treatments_intuitivo_2/assignation_20241116_183458\assigned_treatments_intuitivo2.csv


# MODELO INTUITIVO 3

- Asigna el mejor tratamiento a los 205000 clientes con mejor renta y a los demas 4 correos
- Precios aleatorios también

In [ ]:
df_rut_info_int3 = df_rut_info.copy()
df_probabilities_treatment_int3 = df_probabilities_treatment.copy()
df_cluster_info_int3 = df_cluster_info.copy()
df_simulaciones_info_int_3 = df_simulaciones_info.copy()

In [ ]:
df_estimar_elasticidad_int3 = df_estimar_elasticidad.copy()

In [ ]:
df_rut_info_int3['categoria_clusterizacion_numerica'] = 0

## Precios intuitivo 3

In [ ]:
df_estimar_elasticidad_int3['categoria_clusterizacion_numerica'] = 0

In [ ]:
def function_estimar_elasticidad_int3(df):
    """
    Assigns random prices to clients in a single cluster and estimates the probability 
    of acceptance for each price using logistic regression.

    Parameters:
    - df (pd.DataFrame): DataFrame containing client data with at least 'venta', 
                         'Tasa_Simulado', and 'rut' columns.

    Returns:
    - pd.DataFrame: DataFrame with 'rut', 'price_set', and 'acceptance_probability'.
    """
    
    # Check if the DataFrame is empty
    if df.empty:
        print("The input DataFrame is empty.")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Clean the data: Remove rows with NaN or infinite values in relevant columns
    df_clean = df.replace([np.inf, -np.inf], np.nan).dropna(
        subset=['venta', 'Tasa_Simulado']
    )
    
    # Ensure there are enough data points to fit the model
    if df_clean.shape[0] < 10:
        print("Not enough data to fit the logistic regression model.")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Define the dependent and independent variables
    y = df_clean['venta']
    X = df_clean[['Tasa_Simulado']]
    
    # Add a constant term for the intercept
    X = sm.add_constant(X)
    
    # Fit the logistic regression model
    try:
        logit_model = sm.Logit(y, X)
        result = logit_model.fit(disp=0)
    except Exception as e:
        print(f"Logistic regression failed to converge: {e}")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Determine the range of prices from the data
    tasa_min = df_clean['Tasa_Simulado'].min()
    tasa_max = df_clean['Tasa_Simulado'].max()

    df_clean = df_clean.drop_duplicates(subset='rut')
    
    # Assign a random price to each client within the observed range
    np.random.seed(42)  # For reproducibility; remove or change seed as needed
    df_clean['tasa_optima'] = np.random.uniform(low=tasa_min, high=tasa_max, size=df_clean.shape[0])

    
    # Prepare the data for prediction by adding a constant term
    X_new = sm.add_constant(df_clean['tasa_optima'])
    
    # Predict the acceptance probability using the logistic model
    df_clean['probabilidad_aceptacion_optima'] = result.predict(X_new)
    
    # Ensure probabilities are within [0, 1]
    df_clean['probabilidad_aceptacion_optima'] = df_clean['probabilidad_aceptacion_optima'].clip(0, 1)
    
    # Select the required columns to return
    result_df = df_clean[['rut', 'tasa_optima', 'probabilidad_aceptacion_optima']].copy()
    
    return result_df

In [ ]:
prices3 = function_estimar_elasticidad_int3(df_estimar_elasticidad_int3)

## Tratamientos intuitivo 3

In [ ]:
df_probabilities_treatment_int3 = df_probabilities_treatment_int3[(df_probabilities_treatment_int3['Tratamiento'] == '	Ejecutivo=0, Correos=4')|(df_probabilities_treatment_int3['Tratamiento'] == '	Ejecutivo=1, Correos=2')]

### Estimacion de respuesta

In [ ]:
tratamiento_map = {  # Mapeo de los tratamientos específicos a identificadores numéricos
    "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
}
df_probabilities_int3 = df_probabilities_treatment_int3
df_probabilities_int3['tratamiento_id'] = df_probabilities_int3['Tratamiento'].map(tratamiento_map)

In [ ]:
def function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad, df_simulaciones_info): #df1 es df_estimar_elasticidad y df2 es df_simulaciones_info
    # Paso 1: Preparación de datos y mapeo de clusters
    # Eliminar duplicados en 'df1' para tener un valor único de 'categoria_clusterizacion_numerica' por cada 'rut'.
    df_estimar_elasticidad_unique = df_estimar_elasticidad.drop_duplicates(subset='rut')

    # Crear un mapeo de 'rut' a 'categoria_clusterizacion_numerica' para asociar cada cliente a su cluster numérico.
    rut_cluster_map = df_estimar_elasticidad_unique.set_index('rut')['categoria_clusterizacion_numerica']

    # Mapear la categoría de cluster a cada 'rut' en 'df2' usando el mapeo creado
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['rut'].map(rut_cluster_map)

    # Eliminar filas donde 'categoria_clusterizacion_numerica' es nulo, es decir, aquellos 'rut' sin mapeo de cluster.
    df_simulaciones_info = df_simulaciones_info.dropna(subset=['categoria_clusterizacion_numerica'])

    # Conversión de tipos de datos
    # Convertir 'categoria_clusterizacion_numerica' a entero para garantizar un tipo de dato consistente.
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['categoria_clusterizacion_numerica'].astype(int)

    # Convertir 'simulo' a numérico, reemplazando valores nulos por 0 y asegurando que sea un tipo de dato entero.
    df_simulaciones_info['simulo'] = pd.to_numeric(df_simulaciones_info['simulo'], errors='coerce').fillna(0).astype(int)

    # Convertir 'Tratamiento' a tipo de categoría para optimizar espacio y realizar operaciones categóricas.
    df_simulaciones_info['Tratamiento'] = df_simulaciones_info['Tratamiento'].astype('category')

    # Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número total de registros en cada combinación.
    total_entries_per_cluster_treatment = df_simulaciones_info.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

    # Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
    # Filtrar filas donde 'simulo' es 1 (clientes que realizaron una simulación)
    df_simulations = df_simulaciones_info[df_simulaciones_info['simulo'] == 1]

    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número de registros favorables (simulaciones).
    favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

    # Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
    # Realizar un merge entre 'total_entries_per_cluster_treatment' y 'favorable_entries_per_cluster_treatment' en las columnas de cluster y tratamiento.
    df_probabilities = total_entries_per_cluster_treatment.merge(
        favorable_entries_per_cluster_treatment,
        on=['categoria_clusterizacion_numerica', 'Tratamiento'],
        how='left'
    )

    # Llenar valores nulos en 'caso_favorable' con 0, asegurando que solo las columnas numéricas estén afectadas.
    df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

    # Asegurar que 'caso_total' sea de tipo entero para evitar inconsistencias en los conteos.
    df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

    # Calcular la probabilidad de simulación como el cociente entre 'caso_favorable' y 'caso_total'.
    df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

    # Organizar las columnas del DataFrame resultante para facilitar su análisis.
    df_probabilities = df_probabilities[[
        'categoria_clusterizacion_numerica',
        'Tratamiento',
        'probabilidad_simular',
        'caso_favorable',
        'caso_total'
    ]]

    # Mostrar el DataFrame resultante con la probabilidad de simulación calculada para cada combinación de cluster y tratamiento.
    return df_probabilities


In [ ]:
df_probabilities = function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad_int3, df_simulaciones_info_int_3)

In [ ]:
df_rut_info_int3 = pd.merge(df_rut_info_int3, df_informacion_de_clientes[['rut', 'Renta']], on='rut', how='left')

### Asignación

In [ ]:
# Asignar valor predeterminado de tratamiento 5 a todas las filas
df_rut_info_int3['assigned_treatment'] = 5

# Seleccionar las 205,000 filas con la renta más alta
top_renta_indices = df_rut_info_int3.nlargest(205000, 'Renta').index

# Asignar el valor de tratamiento 8 a estas filas
df_rut_info_int3.loc[top_renta_indices, 'assigned_treatment'] = 8

# Obtener las probabilidades requeridas desde df_probabilities
prob_for_5 = df_probabilities.loc[4, 'probabilidad_simular']  # Para el tratamiento 5
prob_for_8 = df_probabilities.loc[7, 'probabilidad_simular']  # Para el tratamiento 8

# Agregar una nueva columna con probabilidades según el tratamiento asignado
df_rut_info_int3['probabilidad_de_simular'] = df_rut_info_int3['assigned_treatment'].map({
    5: prob_for_5,
    8: prob_for_8
})

In [ ]:
assigned_treatments_int3 = pd.merge(df_rut_info_int3, prices3, on='rut', how='left')

In [ ]:
assigned_treatments_int3.rename(columns={'categoria_clusterizacion_numerica': 'cluster'}, inplace=True)

In [ ]:
assigned_treatments_int3.rename(columns={'probabilidad_aceptacion_optima_x': 'probabilidad_aceptacion_optima', 'tasa_optima_x': 'tasa_optima'}, inplace=True)

In [ ]:
assigned_treatments_int3

,rut,cluster,Probabilidad_No_Pago,tratamientos,probabilidad_aceptacion_optima,tasa_optima,Monto_Simulado_mean,Plazo_Simulado_mean,Plazo_Simulado_min,Plazo_Simulado_max,Plazo_Simulado_mode,Renta,assigned_treatment,probabilidad_de_simular,tasa_optima_y,probabilidad_aceptacion_optima_y
0,1,0,0.028445,"[[0.20384870204179967, 1.0], [0.20784574100201...",0.499757,1.261287,2.672066e+06,27.465434,8.0,48.0,30.0,6.258183e+05,5,0.175640,1.561810,0.383343
1,2,0,0.014320,"[[0.08183039140445127, 1.0], [0.08207471588380...",0.552339,1.219217,7.124037e+05,27.651125,9.0,50.0,30.0,3.172616e+05,5,0.175640,2.426071,0.117775
2,3,0,0.002156,"[[0.20574240514050998, 1.0], [0.21240180638305...",0.624972,1.075107,2.459338e+07,27.514339,9.0,49.0,30.0,1.240551e+07,8,0.366161,2.097991,0.193142
3,4,0,0.034418,"[[0.20302978806049865, 1.0], [0.21238682120531...",0.473732,1.373885,7.612776e+05,27.501192,8.0,48.0,31.0,5.441466e+05,5,0.175640,1.897988,0.254692
4,5,0,0.014978,"[[0.14013414937103366, 1.0], [0.16421079183570...",0.499332,1.265790,4.997373e+05,27.442591,8.0,49.0,30.0,1.870225e+05,5,0.175640,1.234028,0.526982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543646,543647,0,0.037291,"[[0.07533101699690112, 1.0], [0.09850054429890...",0.496040,1.328843,5.326978e+05,27.408767,8.0,52.0,30.0,1.176598e+05,5,0.175640,2.137431,0.182437
543647,543648,0,0.035877,"[[0.20384870204179967, 1.0], [0.20784574100201...",0.499757,1.261287,2.672066e+06,27.465434,8.0,48.0,30.0,1.558612e+06,5,0.175640,1.223011,0.531868
543648,543649,0,0.023306,"[[0.0789925899981393, 1.0], [0.081787777439951...",0.497598,1.307808,4.327878e+06,27.510854,8.0,49.0,30.0,9.449508e+05,5,0.175640,1.562318,0.383129
543649,543650,0,0.015121,"[[0.2075966163542876, 1.0], [0.208343603648015...",0.568717,1.189190,4.125397e+06,27.479876,9.0,49.0,30.0,1.039964e+06,5,0.175640,1.341532,0.479184


In [ ]:
df_cluster_info_int3_csv = df_estimar_elasticidad_int3.groupby('categoria_clusterizacion_numerica').agg({
'Monto_Simulado_mean': 'mean',
'Plazo_Simulado_mean': 'mean',
'Plazo_Simulado_min': 'min',
'Plazo_Simulado_max': 'max',
'Plazo_Simulado_mode': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan})

In [ ]:
df_cluster_info_int3_csv

,Monto_Simulado_mean,Plazo_Simulado_mean,Plazo_Simulado_min,Plazo_Simulado_max,Plazo_Simulado_mode
categoria_clusterizacion_numerica,,,,,
0,6.964109e+06,27.502137,7.0,52.0,30.0


In [ ]:
import os
from datetime import datetime

# Crear una carpeta con un nombre basado en la fecha y hora actual
# 'strftime' genera un timestamp en el formato "YYYYMMDD_HHMMSS" para asegurar nombres de carpeta únicos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"assigned_treatments_intuitivo_3/assignation_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si la carpeta ya existe.

# Definir la ruta del archivo CSV dentro de la nueva carpeta
output_path = os.path.join(folder_name, 'assigned_treatments_intuitivo3.csv')
output_path_cluster = os.path.join(folder_name, 'cluster_info_intuitivo3.csv')

# Guardar el DataFrame 'df_assigned' con las columnas seleccionadas en un archivo CSV
# Se incluyen las columnas clave: 'rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment',
# 'probabilidad_de_simular', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
assigned_treatments_int3[['rut', 'cluster', 'Probabilidad_No_Pago', 'assigned_treatment', 'probabilidad_de_simular', 'tasa_optima',  'probabilidad_aceptacion_optima']].to_csv(output_path, index=False)
df_cluster_info_int3_csv.to_csv(output_path_cluster, index=True)
# Imprimir mensaje de confirmación con la ubicación del archivo CSV guardado
print(f"CSV file saved in folder: {output_path}")

CSV file saved in folder: assigned_treatments_intuitivo_3/assignation_20241116_201041\assigned_treatments_intuitivo3.csv


# MODELO INTUITIVO 4

- Asigna el mejor tratamiento a los 205000 clientes mejor pagadores y a los demas 4 correos
- Precios aleatorios también

In [ ]:
df_rut_info_int4 = df_rut_info.copy()
df_probabilities_treatment_int4 = df_probabilities_treatment.copy()
df_cluster_info_int4 = df_cluster_info.copy()
df_simulaciones_info_int_4 = df_simulaciones_info.copy()

In [ ]:
df_estimar_elasticidad_int4 = df_estimar_elasticidad.copy()

In [ ]:
df_rut_info_int4['categoria_clusterizacion_numerica'] = 0

## Precios intuitivo 4

In [ ]:
df_estimar_elasticidad_int4['categoria_clusterizacion_numerica'] = 0

In [ ]:
def function_estimar_elasticidad_int4(df):
    """
    Assigns random prices to clients in a single cluster and estimates the probability 
    of acceptance for each price using logistic regression.

    Parameters:
    - df (pd.DataFrame): DataFrame containing client data with at least 'venta', 
                         'Tasa_Simulado', and 'rut' columns.

    Returns:
    - pd.DataFrame: DataFrame with 'rut', 'price_set', and 'acceptance_probability'.
    """
    
    # Check if the DataFrame is empty
    if df.empty:
        print("The input DataFrame is empty.")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Clean the data: Remove rows with NaN or infinite values in relevant columns
    df_clean = df.replace([np.inf, -np.inf], np.nan).dropna(
        subset=['venta', 'Tasa_Simulado']
    )
    
    # Ensure there are enough data points to fit the model
    if df_clean.shape[0] < 10:
        print("Not enough data to fit the logistic regression model.")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Define the dependent and independent variables
    y = df_clean['venta']
    X = df_clean[['Tasa_Simulado']]
    
    # Add a constant term for the intercept
    X = sm.add_constant(X)
    
    # Fit the logistic regression model
    try:
        logit_model = sm.Logit(y, X)
        result = logit_model.fit(disp=0)
    except Exception as e:
        print(f"Logistic regression failed to converge: {e}")
        return pd.DataFrame(columns=['rut', 'price_set', 'acceptance_probability'])
    
    # Determine the range of prices from the data
    tasa_min = df_clean['Tasa_Simulado'].min()
    tasa_max = df_clean['Tasa_Simulado'].max()

    df_clean = df_clean.drop_duplicates(subset='rut')
    
    # Assign a random price to each client within the observed range
    np.random.seed(42)  # For reproducibility; remove or change seed as needed
    df_clean['tasa_optima'] = np.random.uniform(low=tasa_min, high=tasa_max, size=df_clean.shape[0])

    
    # Prepare the data for prediction by adding a constant term
    X_new = sm.add_constant(df_clean['tasa_optima'])
    
    # Predict the acceptance probability using the logistic model
    df_clean['probabilidad_aceptacion_optima'] = result.predict(X_new)
    
    # Ensure probabilities are within [0, 1]
    df_clean['probabilidad_aceptacion_optima'] = df_clean['probabilidad_aceptacion_optima'].clip(0, 1)
    
    # Select the required columns to return
    result_df = df_clean[['rut', 'tasa_optima', 'probabilidad_aceptacion_optima']].copy()
    
    return result_df

In [ ]:
prices4 = function_estimar_elasticidad_int4(df_estimar_elasticidad_int4)

## Tratamientos intuitivo 4

In [ ]:
df_probabilities_treatment_int4 = df_probabilities_treatment_int4[(df_probabilities_treatment_int4['Tratamiento'] == '	Ejecutivo=0, Correos=4')|(df_probabilities_treatment_int4['Tratamiento'] == 'Ejecutivo=1, Correos=2')]

### Estimacion de respuesta

In [ ]:
tratamiento_map = {  # Mapeo de los tratamientos específicos a identificadores numéricos
    "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
}
df_probabilities_int4 = df_probabilities_treatment_int4
df_probabilities_int4['tratamiento_id'] = df_probabilities_int4['Tratamiento'].map(tratamiento_map)

In [ ]:
def function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad, df_simulaciones_info): #df1 es df_estimar_elasticidad y df2 es df_simulaciones_info
    # Paso 1: Preparación de datos y mapeo de clusters
    # Eliminar duplicados en 'df1' para tener un valor único de 'categoria_clusterizacion_numerica' por cada 'rut'.
    df_estimar_elasticidad_unique = df_estimar_elasticidad.drop_duplicates(subset='rut')

    # Crear un mapeo de 'rut' a 'categoria_clusterizacion_numerica' para asociar cada cliente a su cluster numérico.
    rut_cluster_map = df_estimar_elasticidad_unique.set_index('rut')['categoria_clusterizacion_numerica']

    # Mapear la categoría de cluster a cada 'rut' en 'df2' usando el mapeo creado
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['rut'].map(rut_cluster_map)

    # Eliminar filas donde 'categoria_clusterizacion_numerica' es nulo, es decir, aquellos 'rut' sin mapeo de cluster.
    df_simulaciones_info = df_simulaciones_info.dropna(subset=['categoria_clusterizacion_numerica'])

    # Conversión de tipos de datos
    # Convertir 'categoria_clusterizacion_numerica' a entero para garantizar un tipo de dato consistente.
    df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['categoria_clusterizacion_numerica'].astype(int)

    # Convertir 'simulo' a numérico, reemplazando valores nulos por 0 y asegurando que sea un tipo de dato entero.
    df_simulaciones_info['simulo'] = pd.to_numeric(df_simulaciones_info['simulo'], errors='coerce').fillna(0).astype(int)

    # Convertir 'Tratamiento' a tipo de categoría para optimizar espacio y realizar operaciones categóricas.
    df_simulaciones_info['Tratamiento'] = df_simulaciones_info['Tratamiento'].astype('category')

    # Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número total de registros en cada combinación.
    total_entries_per_cluster_treatment = df_simulaciones_info.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

    # Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
    # Filtrar filas donde 'simulo' es 1 (clientes que realizaron una simulación)
    df_simulations = df_simulaciones_info[df_simulaciones_info['simulo'] == 1]

    # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número de registros favorables (simulaciones).
    favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

    # Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
    # Realizar un merge entre 'total_entries_per_cluster_treatment' y 'favorable_entries_per_cluster_treatment' en las columnas de cluster y tratamiento.
    df_probabilities = total_entries_per_cluster_treatment.merge(
        favorable_entries_per_cluster_treatment,
        on=['categoria_clusterizacion_numerica', 'Tratamiento'],
        how='left'
    )

    # Llenar valores nulos en 'caso_favorable' con 0, asegurando que solo las columnas numéricas estén afectadas.
    df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

    # Asegurar que 'caso_total' sea de tipo entero para evitar inconsistencias en los conteos.
    df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

    # Calcular la probabilidad de simulación como el cociente entre 'caso_favorable' y 'caso_total'.
    df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

    # Organizar las columnas del DataFrame resultante para facilitar su análisis.
    df_probabilities = df_probabilities[[
        'categoria_clusterizacion_numerica',
        'Tratamiento',
        'probabilidad_simular',
        'caso_favorable',
        'caso_total'
    ]]

    # Mostrar el DataFrame resultante con la probabilidad de simulación calculada para cada combinación de cluster y tratamiento.
    return df_probabilities


In [ ]:
df_probabilities = function_estimar_respuesta_a_tratamiento(df_estimar_elasticidad_int4, df_simulaciones_info_int_4)

In [ ]:
df_rut_info_int4 = pd.merge(df_rut_info_int4, df_informacion_de_clientes[['rut', 'Probabilidad_No_Pago']], on='rut', how='left')

In [ ]:
df_rut_info_int4.rename(columns={'Probabilidad_No_Pago_x': 'Probabilidad_No_Pago'}, inplace = True)

### Asignación

In [ ]:
# Asignar valor predeterminado de tratamiento 5 a todas las filas
df_rut_info_int4['assigned_treatment'] = 5

# Seleccionar las 205,000 filas con la Probabilidad_No_Pago mas baja
top_pnp_indices = df_rut_info_int4.nsmallest(205000, 'Probabilidad_No_Pago').index

# Asignar el valor de tratamiento 8 a estas filas
df_rut_info_int4.loc[top_pnp_indices, 'assigned_treatment'] = 8

# Obtener las probabilidades requeridas desde df_probabilities
prob_for_5 = df_probabilities.loc[4, 'probabilidad_simular']  # Para el tratamiento 5
prob_for_8 = df_probabilities.loc[7, 'probabilidad_simular']  # Para el tratamiento 8

# Agregar una nueva columna con probabilidades según el tratamiento asignado
df_rut_info_int4['probabilidad_de_simular'] = df_rut_info_int4['assigned_treatment'].map({
    5: prob_for_5,
    8: prob_for_8
})

In [ ]:
assigned_treatments_int4 = pd.merge(df_rut_info_int4, prices4, on='rut', how='left')

In [ ]:
assigned_treatments_int4.rename(columns={'categoria_clusterizacion_numerica': 'cluster'}, inplace=True)

In [ ]:
assigned_treatments_int4.rename(columns={'probabilidad_aceptacion_optima_x': 'probabilidad_aceptacion_optima', 'tasa_optima_x': 'tasa_optima'}, inplace=True)

In [ ]:
assigned_treatments_int4

,rut,cluster,Probabilidad_No_Pago,tratamientos,probabilidad_aceptacion_optima,tasa_optima,Monto_Simulado_mean,Plazo_Simulado_mean,Plazo_Simulado_min,Plazo_Simulado_max,Plazo_Simulado_mode,Probabilidad_No_Pago_y,assigned_treatment,probabilidad_de_simular,tasa_optima_y,probabilidad_aceptacion_optima_y
0,1,0,0.028445,"[[0.20384870204179967, 1.0], [0.20784574100201...",0.499757,1.261287,2.672066e+06,27.465434,8.0,48.0,30.0,0.028445,5,0.175640,1.561810,0.383343
1,2,0,0.014320,"[[0.08183039140445127, 1.0], [0.08207471588380...",0.552339,1.219217,7.124037e+05,27.651125,9.0,50.0,30.0,0.014320,5,0.175640,2.426071,0.117775
2,3,0,0.002156,"[[0.20574240514050998, 1.0], [0.21240180638305...",0.624972,1.075107,2.459338e+07,27.514339,9.0,49.0,30.0,0.002156,8,0.366161,2.097991,0.193142
3,4,0,0.034418,"[[0.20302978806049865, 1.0], [0.21238682120531...",0.473732,1.373885,7.612776e+05,27.501192,8.0,48.0,31.0,0.034418,5,0.175640,1.897988,0.254692
4,5,0,0.014978,"[[0.14013414937103366, 1.0], [0.16421079183570...",0.499332,1.265790,4.997373e+05,27.442591,8.0,49.0,30.0,0.014978,5,0.175640,1.234028,0.526982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543646,543647,0,0.037291,"[[0.07533101699690112, 1.0], [0.09850054429890...",0.496040,1.328843,5.326978e+05,27.408767,8.0,52.0,30.0,0.037291,5,0.175640,2.137431,0.182437
543647,543648,0,0.035877,"[[0.20384870204179967, 1.0], [0.20784574100201...",0.499757,1.261287,2.672066e+06,27.465434,8.0,48.0,30.0,0.035877,5,0.175640,1.223011,0.531868
543648,543649,0,0.023306,"[[0.0789925899981393, 1.0], [0.081787777439951...",0.497598,1.307808,4.327878e+06,27.510854,8.0,49.0,30.0,0.023306,5,0.175640,1.562318,0.383129
543649,543650,0,0.015121,"[[0.2075966163542876, 1.0], [0.208343603648015...",0.568717,1.189190,4.125397e+06,27.479876,9.0,49.0,30.0,0.015121,5,0.175640,1.341532,0.479184


In [ ]:
df_cluster_info_int4_csv = df_estimar_elasticidad_int4.groupby('categoria_clusterizacion_numerica').agg({
'Monto_Simulado_mean': 'mean',
'Plazo_Simulado_mean': 'mean',
'Plazo_Simulado_min': 'min',
'Plazo_Simulado_max': 'max',
'Plazo_Simulado_mode': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan})

In [ ]:
df_cluster_info_int4_csv

,Monto_Simulado_mean,Plazo_Simulado_mean,Plazo_Simulado_min,Plazo_Simulado_max,Plazo_Simulado_mode
categoria_clusterizacion_numerica,,,,,
0,6.964109e+06,27.502137,7.0,52.0,30.0


In [ ]:
import os
from datetime import datetime

# Crear una carpeta con un nombre basado en la fecha y hora actual
# 'strftime' genera un timestamp en el formato "YYYYMMDD_HHMMSS" para asegurar nombres de carpeta únicos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"assigned_treatments_intuitivo_4/assignation_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si la carpeta ya existe.

# Definir la ruta del archivo CSV dentro de la nueva carpeta
output_path = os.path.join(folder_name, 'assigned_treatments_intuitivo4.csv')
output_path_cluster = os.path.join(folder_name, 'cluster_info_intuitivo4.csv')

# Guardar el DataFrame 'df_assigned' con las columnas seleccionadas en un archivo CSV
# Se incluyen las columnas clave: 'rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment',
# 'probabilidad_de_simular', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
assigned_treatments_int4[['rut', 'cluster', 'Probabilidad_No_Pago', 'assigned_treatment', 'probabilidad_de_simular', 'tasa_optima',  'probabilidad_aceptacion_optima']].to_csv(output_path, index=False)
df_cluster_info_int4_csv.to_csv(output_path_cluster, index=True)
# Imprimir mensaje de confirmación con la ubicación del archivo CSV guardado
print(f"CSV file saved in folder: {output_path}")

CSV file saved in folder: assigned_treatments_intuitivo_4/assignation_20241116_212213\assigned_treatments_intuitivo4.csv


# 7. RL

## Definicion de la clase

In [46]:
# Importar las librerías necesarias
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import statsmodels.api as sm
from gurobipy import Model, GRB, quicksum
from stable_baselines3 import DQN
from stable_baselines3.common.env_checker import check_env
from datetime import datetime
import os
import gc
import logging  # Importar el módulo logging
from sklearn.preprocessing import normalize

# Configuración básica del logging
logging.basicConfig(
    level=logging.INFO,  # Nivel de logging (puede ser DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  # Mostrar logs en la consola
        # Puedes agregar FileHandler para guardar logs en un archivo si lo deseas
    ]
)
logger = logging.getLogger(__name__)  # Crear un logger

# Definir la clase ClusteringEnv
class ClusteringEnv(gym.Env):
    def __init__(self, data, df_sim_ventas_tratamiento, df_simulaciones_info):
        super(ClusteringEnv, self).__init__()
        
        logger.info("Inicializando ClusteringEnv...")
        
        # Datos de entrada
        self.data = data
        self.df_sim_ventas_tratamiento = df_sim_ventas_tratamiento
        self.df_simulaciones_info = df_simulaciones_info
        
        # Definir variables excluyendo 'rut'
        self.variables = list(self.data.columns)
        if 'rut' in self.variables:
            self.variables.remove('rut')
        logger.debug(f"Variables excluidas: {self.variables}")
        
        # Identificar variables categóricas y continuas
        self.categorical_vars = self.data.select_dtypes(include=['object', 'category']).columns.tolist()
        self.continuous_vars = self.data.select_dtypes(include=[np.number]).columns.tolist()
        if 'rut' in self.continuous_vars:
            self.continuous_vars.remove('rut')
        logger.info(f"Variables categóricas: {self.categorical_vars}")
        logger.info(f"Variables continuas: {self.continuous_vars}")
        
        # Parámetros de cortes para variables continuas
        self.max_splits = 3  # Número máximo de cortes
        self.min_splits = 2  # Número mínimo de cortes
        logger.info(f"Parámetros de splits - Max: {self.max_splits}, Min: {self.min_splits}")
        
        # Penalización por número de clusters
        self.penalty_factor = 20000000  # Ajusta este valor según tus necesidades
        self.executives_remaining = 0
        logger.debug(f"Factor de penalización por cluster: {self.penalty_factor}")
        
        # Crear la lista de acciones
        self.action_list = self.create_action_list()
        logger.info(f"Número de acciones posibles: {len(self.action_list)}")
        
        # Inicializar el estado y otros atributos necesarios
        self.reset()  # Esto inicializa self.included_vars y self.splits
        
        # Definir el espacio de acciones y observaciones
        self.action_space = spaces.Discrete(len(self.action_list))
        state_size = self.get_state().shape[0]
        self.observation_space = spaces.Box(low=0, high=1, shape=(state_size,), dtype=np.float32)
        logger.debug(f"Tamaño del estado: {state_size}")
        
        # Inicializar el contador de pasos
        self.current_step = 0
        self.max_steps = 50  # Número máximo de pasos por episodio
        logger.info("ClusteringEnv inicializado correctamente.")
    
    def reset(self, seed=None, options=None):
        # Reiniciar el entorno al estado inicial
        logger.info("Reiniciando el entorno...")
        self.executives_remaining = 0
        self.included_vars = {var: 0 for var in self.variables}  # Inicialmente, todas las variables excluidas
        self.splits = {var: [] for var in self.continuous_vars}  # Inicialmente, sin cortes
        self.state = self.get_state()
        self.current_step = 0
        logger.info(f"Estado reiniciado: {self.state}")
        return self.state, {}
    
    def get_state(self):
        # Representar el estado actual como un array
        state = []
        for var in self.variables:
            included = self.included_vars[var]
            state.append(included)
            if var in self.continuous_vars and included:
                splits = self.splits[var]
                min_val = self.data[var].min()
                max_val = self.data[var].max()
                if max_val == min_val:
                    normalized_splits = [0] * self.max_splits
                else:
                    normalized_splits = [(s - min_val) / (max_val - min_val) for s in splits]
                    # Rellenar con ceros para tener longitud fija
                    normalized_splits += [0] * (self.max_splits - len(normalized_splits))
                    normalized_splits = normalized_splits[:self.max_splits]
                state.extend(normalized_splits)
            elif var in self.continuous_vars:
                # Si la variable no está incluida, agregar ceros
                state.extend([0] * self.max_splits)
        return np.array(state, dtype=np.float32)
    
    def create_action_list(self):
        actions = []
        # Acciones para incluir o excluir variables
        for var in self.variables:
            actions.append(('toggle_variable', var, {}))
        # Acciones para ajustar los cortes de variables continuas
        for var in self.continuous_vars:
            # Aumentar número de cortes
            actions.append(('adjust_splits', var, {'operation': 'increase'}))
            # Disminuir número de cortes
            actions.append(('adjust_splits', var, {'operation': 'decrease'}))
            # Mover cada punto de corte hacia arriba o abajo
            for index in range(self.max_splits):
                actions.append(('adjust_splits', var, {'operation': 'move', 'index': index, 'amount': +1}))
                actions.append(('adjust_splits', var, {'operation': 'move', 'index': index, 'amount': -1}))
        print(actions)
        return actions
    
    def step(self, action_index):
        logger.debug(f"Ejecutando acción: {self.action_list[action_index]}")
        # Aplicar la acción al estado actual
        action = self.action_list[action_index]
        self.apply_action(action)
        
        # Recalcular los clusters
        logger.info("Recalculando clusters...")
        df_clusters = self.perform_clustering()
        
        # Recalcular las métricas y obtener la recompensa
        try:
            total_revenue, num_clusters = self.recalculate_metrics(df_clusters)
            reward = total_revenue - self.penalty_factor * num_clusters - int(self.executives_remaining) * 10000
            logger.debug(f"Recompensa calculada: {reward} (Revenue: {total_revenue}, Clusters: {num_clusters})")
        except Exception as e:
            # En caso de error, asignar una recompensa negativa
            logger.error(f"Error en recalculate_metrics: {e}")
            reward = -1000
            total_revenue = 0
            num_clusters = 0
        
        # Incrementar el contador de pasos
        self.current_step += 1
        done = self.current_step >= self.max_steps
        
        # En Gymnasium, también se debe definir `truncated`. Aquí lo dejamos como False.
        truncated = False
        
        # Actualizar el estado
        self.state = self.get_state()
        
        # Crear un diccionario de información
        info = {
            'total_revenue': total_revenue,
            'num_clusters': num_clusters
        }
        
        logger.debug(f"Estado actual: {self.state}")
        logger.debug(f"Paso {self.current_step}/{self.max_steps} completado.")

        # Limpiar referencias a df_clusters y métricas después de usarlas
        del df_clusters, total_revenue, num_clusters
        gc.collect()
                
        # Retornar el nuevo estado, recompensa, done, truncated, y el diccionario de info
        return self.state, reward, done, truncated, info
    
    def apply_action(self, action):
        # Aplicar la acción al estado actual
        action_type, var, params = action
        logger.info(f"Aplicando acción tipo: {action_type} sobre variable: {var} con parámetros: {params}")
        
        if action_type == 'toggle_variable':
            # Incluir o excluir la variable
            self.included_vars[var] = 1 - self.included_vars[var]
            logger.info(f"Variable '{var}' incluida: {self.included_vars[var]}")
            if var in self.continuous_vars and self.included_vars[var]:
                # Inicializar cortes si la variable es incluida
                if not self.splits[var]:
                    self.splits[var] = self.initialize_splits(var)
                    logger.info(f"Cortes inicializados para variable '{var}': {self.splits[var]}")
                    
        elif action_type == 'adjust_splits':
            # Verificar que la acción es válida
            if var in self.continuous_vars and self.included_vars[var]:
                operation = params.get('operation')
                index = params.get('index', 0)
                amount = params.get('amount', 0)
                
                if operation == 'increase':
                    if len(self.splits[var]) < self.max_splits:
                        self.add_split(var)
                        logger.info(f"Corte aumentado para variable '{var}': {self.splits[var]}")
                    elif len(self.splits[var]) == self.max_splits:
                        logger.warning(f"No se puede aumentar el número de cortes para '{var}': ya se alcanzó el máximo.")
                        logger.info(f"Se disminuirá el número de cortes para '{var}' en su lugar.")
                        removed_split = self.splits[var].pop()
                        logger.info(f"Corte disminuido para variable '{var}', eliminado: {removed_split}")

                        
                elif operation == 'decrease':
                    if len(self.splits[var]) > self.min_splits:
                        removed_split = self.splits[var].pop()
                        logger.info(f"Corte disminuido para variable '{var}', eliminado: {removed_split}")
                    elif len(self.splits[var]) == self.min_splits:
                        logger.warning(f"No se puede disminuir el número de cortes para '{var}': ya se alcanzó el mínimo.")
                        logger.info(f"Se aumentará el número de cortes para '{var}' en su lugar.")
                        self.add_split(var)
                        
                elif operation == 'move':
                    self.move_split(var, index, amount)
                    logger.info(f"Corte movido para variable '{var}': {self.splits[var]}")
            else:
                # Si no es válida, elegir una nueva acción
                logger.warning(f"No se puede ajustar separaciones para '{var}': no es continua o no está incluida en la clusterización.")
                new_action_index = np.random.choice(range(len(self.action_list)))  # Seleccionar una acción aleatoria
                logger.info(f"Seleccionando nueva acción: {self.action_list[new_action_index]}")
                self.apply_action(self.action_list[new_action_index])  # Aplicar la nueva acción
                return  # Salir para evitar ajustes adicionales
                
        else:
            logger.warning(f"Acción desconocida: {action_type}")
        
        # Asegurar que los cortes están ordenados y dentro del rango
        for var_cont in self.continuous_vars:
            if self.splits[var_cont]:
                old_splits = self.splits[var_cont].copy()
                self.splits[var_cont] = sorted(set(self.splits[var_cont]))
                min_val = self.data[var_cont].min()
                max_val = self.data[var_cont].max()
                self.splits[var_cont] = [s for s in self.splits[var_cont] if min_val < s < max_val]
                if self.splits[var_cont] != old_splits:
                    logger.debug(f"Cortes ajustados para variable '{var_cont}': {self.splits[var_cont]}")
        
    def initialize_splits(self, var):
        # Inicializar cortes en cuantiles equidistantes
        min_val = self.data[var].min()
        max_val = self.data[var].max()
        num_splits = self.min_splits
        if num_splits == 0:
            return []
        splits = np.linspace(min_val, max_val, num_splits + 1)[1:-1].tolist()
        return splits
    
    def add_split(self, var):
        # Agregar un nuevo corte en el punto medio del intervalo más grande
        splits = self.splits[var]
        if not splits:
            min_val = self.data[var].min()
            max_val = self.data[var].max()
            new_split = (min_val + max_val) / 2
            logger.debug(f"Agregando primer split para '{var}': {new_split}")
        else:
            intervals = [self.data[var].min()] + splits + [self.data[var].max()]
            max_interval = 0
            max_idx = 0
            for i in range(len(intervals) - 1):
                interval = intervals[i+1] - intervals[i]
                if interval > max_interval:
                    max_interval = interval
                    max_idx = i
            new_split = (intervals[max_idx] + intervals[max_idx+1]) / 2
            logger.debug(f"Agregando split en el intervalo más grande para '{var}': {new_split}")
        splits.append(new_split)
        self.splits[var] = splits
    
    def move_split(self, var, index, amount):
        # Mover el corte en 'index' una cantidad 'amount' en términos de cuantiles
        if 0 <= index < len(self.splits[var]):
            quantile_steps = np.arange(0, 1.05, 0.05)  # 0%, 5%, ..., 100%
            quantile_values = self.data[var].quantile(quantile_steps)
            current_split = self.splits[var][index]
            # Encontrar el índice de cuantil más cercano al split actual
            current_quantile_index = np.argmin(np.abs(quantile_values.values - current_split))
            new_quantile_index = current_quantile_index + amount  # Mover 'amount' pasos de 5% cuantiles
            # Asegurar que el índice esté dentro del rango
            new_quantile_index = max(0, min(new_quantile_index, len(quantile_values) - 1))
            old_split = self.splits[var][index]
            new_split = quantile_values.values[new_quantile_index]
            self.splits[var][index] = new_split
            logger.debug(f"Moviendo split para '{var}' en índice {index}: de {old_split} a {self.splits[var][index]}")

    def perform_clustering(self):
        logger.info("Realizando clustering...")
        # Realizar el clustering basado en la configuración actual
        df = self.data.copy()
        for var in self.variables:
            logger.info(f"Procesando variable: {var}")
            if self.included_vars[var]:
                if var in self.categorical_vars:
                    df[var + '_cluster'] = df[var]
                    logger.info(f"Variable categórica '{var}' separada en los grupos: {df[var + '_cluster'].unique()}")
                elif var in self.continuous_vars:
                    splits = self.splits[var]
                    if not splits:
                        # Si no hay cortes, asignar todos al mismo bin
                        df[var + '_cluster'] = 0
                    else:
                        bins = [-np.inf] + splits + [np.inf]
                        labels = [f'{var}_bin_{i}' for i in range(len(bins)-1)]
                        df[var + '_cluster'] = pd.cut(df[var], bins=bins, labels=labels)
                        logger.info(f"Cortes para variable '{var}': {splits}")
        # Combinar las variables de cluster para formar la categoría de clusterización
        cluster_vars = [var + '_cluster' for var in self.variables if self.included_vars[var]]
        logger.info(f"Variables incluidas en clusterización: {cluster_vars}")
        if cluster_vars:
            df['categoria_clusterizacion'] = df[cluster_vars].astype(str).agg(' '.join, axis=1)
            df['categoria_clusterizacion_numerica'] = df['categoria_clusterizacion'].astype('category').cat.codes
            logger.debug(f"Clusters formados: {df['categoria_clusterizacion_numerica'].nunique()}")
        else:
            # Si no hay variables incluidas, asignar a un solo cluster
            df['categoria_clusterizacion_numerica'] = 0
            logger.debug("No se incluyeron variables, todos los datos asignados al cluster 0.")
        self.current_clusters = df[['rut', 'categoria_clusterizacion_numerica']]
        num_clusters = df['categoria_clusterizacion_numerica'].nunique()
        logger.info(f"Variables incluidas en este clustering: {cluster_vars}")
        logger.info(f"Clustering completado. Número de clusters: {num_clusters}")
        return df
    
    def function_estimar_elasticidad(self, df_estimar_elasticidad):
        logger.info("Estimando elasticidad...")
        # Inicializar listas para almacenar resultados globales de revenue, clientes, créditos y simulaciones
        lista_revenue = []
        lista_clientes = []
        lista_creditos = []
        lista_simulaciones = []

        cluster_results = []  # Lista para almacenar resultados específicos de cada cluster

        # Obtener los números únicos de cada cluster
        cluster_numbers = df_estimar_elasticidad['categoria_clusterizacion_numerica'].unique()
        logger.debug(f"Número de clusters para estimar elasticidad: {len(cluster_numbers)}")

        # Iterar sobre cada cluster identificado por 'categoria_clusterizacion_numerica'
        for cluster_num in cluster_numbers:
            logger.debug(f"Procesando cluster {cluster_num}...")
            # Filtrar los datos correspondientes al cluster actual
            df_cluster = df_estimar_elasticidad[df_estimar_elasticidad['categoria_clusterizacion_numerica'] == cluster_num]
            
            # Asegurarse de que existen datos para ambos casos: venta == 1 y venta == 0
            if df_cluster.empty or df_cluster['venta'].isnull().all():
                logger.warning(f"Cluster {cluster_num} está vacío o todas las ventas son nulas. Se salta.")
                continue  # Saltar este cluster si no cumple con la condición
            
            # Remover filas donde 'venta' o 'Tasa_Simulado' son nulos o infinitos
            df_cluster = df_cluster.replace([np.inf, -np.inf], np.nan)
            df_cluster = df_cluster.dropna(subset=['venta', 'Tasa_Simulado', 'Plazo_Simulado', 'Monto_Simulado', 'Probabilidad_No_Pago'])
            
            # Saltar el cluster si no hay suficientes puntos de datos
            if df_cluster.shape[0] < 10:
                logger.warning(f"Cluster {cluster_num} tiene menos de 10 registros después de limpiar. Se salta.")
                continue
            
            # Extraer las variables 'venta' (como variable dependiente) y 'Tasa_Simulado' (como predictor)
            y = df_cluster['venta']
            X = df_cluster[['Tasa_Simulado']]
            
            # Añadir un término constante para el intercepto
            X = sm.add_constant(X)
            
            # Remover filas con valores NaN o Inf en X o y
            is_finite = np.isfinite(X).all(1) & np.isfinite(y)
            X = X[is_finite]
            y = y[is_finite]
            
            # Asegurarse de que después de remover NaN/Inf, todavía hay suficientes datos
            if len(y) < 10:
                logger.warning(f"Cluster {cluster_num} tiene menos de 10 registros después de filtrar finitos. Se salta.")
                continue
            
            # Ajustar el modelo de regresión logística
            logit_model = sm.Logit(y, X)
            try:
                result = logit_model.fit(disp=0)
                logger.debug(f"Modelo ajustado para cluster {cluster_num}.")
            except Exception as e:
                logger.error(f"No se pudo ajustar el modelo para el cluster {cluster_num}: {e}")
                del df_cluster, y, X, logit_model, result  # Eliminar referencias
                gc.collect()
                continue  # Saltar el cluster si el modelo no converge
            
            # Crear una cuadrícula de valores de 'Tasa_Simulado' para predicciones
            tasa_min = df_cluster['Tasa_Simulado'].min()
            tasa_max = df_cluster['Tasa_Simulado'].max()
            tasas_grid = np.linspace(tasa_min, tasa_max, 105)
            
            # Predecir la probabilidad de aceptación usando el modelo ajustado
            X_grid = sm.add_constant(tasas_grid)
            acceptance_probability = result.predict(X_grid)
            
            # Asegurar que las probabilidades están en el rango [0, 1]
            acceptance_probability = np.clip(acceptance_probability, 0, 1)
            
            # Calcular valores medios necesarios para el cálculo de revenue
            n = df_cluster['Plazo_Simulado'].mean()
            vp = df_cluster['Monto_Simulado'].mean()
            pnp = df_cluster['Probabilidad_No_Pago'].mean()
            data = {
                'Plazo_Simulado_medio': n, 
                'Monto_Simulado_medio': vp, 
                'Probabilidad_No_Pago_media': pnp
            }
            
            # Calcular el revenue potencial
            i = tasas_grid / 100  # Convertir a decimal
            one_plus_i_pow_n = np.power(1 + i, n)
            annuity_factor = (i * one_plus_i_pow_n) / (one_plus_i_pow_n - 1)
            revenue = (n * vp * annuity_factor) - vp
            potential_revenue = revenue * (1 - pnp)
            
            # Calcular el promedio de simulaciones por fecha
            df_cluster_simulaciones_1 = df_cluster[df_cluster['simulo'] == 1]
            num_dates = df_cluster_simulaciones_1['fecha'].nunique()
            total_simulaciones = df_cluster_simulaciones_1['simulo'].sum()
            simulaciones_medias = total_simulaciones / num_dates if num_dates else 0
            
            # Saltar el cluster si no hay simulaciones
            if simulaciones_medias == 0:
                logger.warning(f"Cluster {cluster_num} no tiene simulaciones medias. Se salta.")
                continue
            
            # Calcular el revenue esperado
            expected_revenue = acceptance_probability * potential_revenue * simulaciones_medias
            
            # Encontrar la tasa que maximiza el revenue esperado
            idx_max = np.argmax(expected_revenue)
            max_price = tasas_grid[idx_max]
            max_expected_revenue = expected_revenue[idx_max]
            
            # Probabilidad de aceptación en la tasa óptima
            prob_aceptacion_optima = acceptance_probability[idx_max]
            
            # Número esperado de créditos aceptados
            num_creditos_aceptados = round(prob_aceptacion_optima * simulaciones_medias)
            
            # Número de clientes únicos en el cluster
            num_clients = df_cluster['rut'].nunique()
            
            # Imprimir resultados para cada cluster
            logger.info(f'Cluster {cluster_num}:')
            logger.info(f'- Precio Máx. Revenue Esperado = {max_price:.2f}%')
            logger.info(f'- Revenue Esperado Máximo = {max_expected_revenue:,.2f}')
            logger.info(f'- Número de clientes en el cluster = {num_clients}')
            logger.info(f'- Número de simulaciones en el cluster = {simulaciones_medias:.2f}')
            logger.info(f'- Probabilidad de aceptación en el precio óptimo = {prob_aceptacion_optima:.4f}')
            logger.info(f'- Número esperado de créditos aceptados = {num_creditos_aceptados}')
            logger.info(f'- Monto medio simulado = {data["Monto_Simulado_medio"]:,.2f}')
            logger.info(f'- Plazo medio simulado = {data["Plazo_Simulado_medio"]:,.2f}')
            logger.info(f'- Probabilidad de no pago media = {data["Probabilidad_No_Pago_media"]:.4f}\n')
            
            # Agregar resultados a las listas globales
            lista_clientes.append(num_clients)
            lista_revenue.append(max_expected_revenue)
            lista_creditos.append(num_creditos_aceptados)
            lista_simulaciones.append(simulaciones_medias)
            
            # Almacenar resultados por cluster en cluster_results
            cluster_results.append({
                'categoria_clusterizacion_numerica': cluster_num,
                'tasa_optima': max_price,
                'probabilidad_aceptacion_optima': prob_aceptacion_optima,
                'revenue_esperado_maximo': max_expected_revenue,
                'numero_clientes': num_clients,
                'numero_simulaciones_medias': simulaciones_medias,
                'numero_creditos_esperados': num_creditos_aceptados,
                'monto_medio_simulado': data["Monto_Simulado_medio"],
                'plazo_medio_simulado': data["Plazo_Simulado_medio"],
                'probabilidad_no_pago_media': data["Probabilidad_No_Pago_media"]
            })

            del df_cluster, y, X, logit_model, result, X_grid, acceptance_probability, expected_revenue
            gc.collect()
        # Imprimir resultados globales
        total_revenue = sum(lista_revenue)
        total_clientes = sum(lista_clientes)
        total_simulaciones = sum(lista_simulaciones)
        total_creditos = sum(lista_creditos)

        logger.info(f"El revenue total esperado es: {total_revenue:,.2f} con un total de {total_clientes} clientes, "
                    f"{total_simulaciones:,.2f} simulaciones, y {total_creditos} créditos.")

        # Crear un DataFrame a partir de cluster_results
        df_cluster_results = pd.DataFrame(cluster_results)

        # Incorporar los resultados por cluster de 'df_cluster_results' a 'df_estimar_elasticidad'
        df_estimar_elasticidad = df_estimar_elasticidad.merge(
            df_cluster_results[['categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']],
            on='categoria_clusterizacion_numerica', 
            how='left'
        )

        return {'df_estimar_elasticidad': df_estimar_elasticidad, 'total_revenue': total_revenue, 'total_clientes': total_clientes, 'total_simulaciones': total_simulaciones, 'total_creditos': total_creditos}
    
    def function_estimar_respuesta_a_tratamiento(self, df_estimar_elasticidad, df_simulaciones_info): #df1 es df_estimar_elasticidad y df2 es df_simulaciones_info
        # Paso 1: Preparación de datos y mapeo de clusters
        # Eliminar duplicados en 'df1' para tener un valor único de 'categoria_clusterizacion_numerica' por cada 'rut'.
        df_estimar_elasticidad_unique = df_estimar_elasticidad.drop_duplicates(subset='rut')

        # Crear un mapeo de 'rut' a 'categoria_clusterizacion_numerica' para asociar cada cliente a su cluster numérico.
        rut_cluster_map = df_estimar_elasticidad_unique.set_index('rut')['categoria_clusterizacion_numerica']

        # Mapear la categoría de cluster a cada 'rut' en 'df2' usando el mapeo creado
        df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['rut'].map(rut_cluster_map)

        # Eliminar filas donde 'categoria_clusterizacion_numerica' es nulo, es decir, aquellos 'rut' sin mapeo de cluster.
        df_simulaciones_info = df_simulaciones_info.dropna(subset=['categoria_clusterizacion_numerica'])

        # Conversión de tipos de datos
        # Convertir 'categoria_clusterizacion_numerica' a entero para garantizar un tipo de dato consistente.
        df_simulaciones_info['categoria_clusterizacion_numerica'] = df_simulaciones_info['categoria_clusterizacion_numerica'].astype(int)

        # Convertir 'simulo' a numérico, reemplazando valores nulos por 0 y asegurando que sea un tipo de dato entero.
        df_simulaciones_info['simulo'] = pd.to_numeric(df_simulaciones_info['simulo'], errors='coerce').fillna(0).astype(int)

        # Convertir 'Tratamiento' a tipo de categoría para optimizar espacio y realizar operaciones categóricas.
        df_simulaciones_info['Tratamiento'] = df_simulaciones_info['Tratamiento'].astype('category')

        # Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
        # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número total de registros en cada combinación.
        total_entries_per_cluster_treatment = df_simulaciones_info.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

        # Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
        # Filtrar filas donde 'simulo' es 1 (clientes que realizaron una simulación)
        df_simulations = df_simulaciones_info[df_simulaciones_info['simulo'] == 1]

        # Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número de registros favorables (simulaciones).
        favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

        # Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
        # Realizar un merge entre 'total_entries_per_cluster_treatment' y 'favorable_entries_per_cluster_treatment' en las columnas de cluster y tratamiento.
        df_probabilities = total_entries_per_cluster_treatment.merge(
            favorable_entries_per_cluster_treatment,
            on=['categoria_clusterizacion_numerica', 'Tratamiento'],
            how='left'
        )

        # Llenar valores nulos en 'caso_favorable' con 0, asegurando que solo las columnas numéricas estén afectadas.
        df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

        # Asegurar que 'caso_total' sea de tipo entero para evitar inconsistencias en los conteos.
        df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

        # Calcular la probabilidad de simulación como el cociente entre 'caso_favorable' y 'caso_total'.
        df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

        # Organizar las columnas del DataFrame resultante para facilitar su análisis.
        df_probabilities = df_probabilities[[
            'categoria_clusterizacion_numerica',
            'Tratamiento',
            'probabilidad_simular',
            'caso_favorable',
            'caso_total'
        ]]

        logger.info("Estimación de respuesta a tratamientos completada.")
        return df_probabilities

    def function_modelo_asignacion_tratamientos(self, df_cluster_info, df_probabilities_treatment, df_rut_info, costo_sms, capacidad_ejecutivos):
        logger.info("Iniciando modelo de asignación de tratamientos...")
        try:
            # Definir el mapeo de tratamientos
            tratamiento_map = {  # Mapeo de los tratamientos específicos a identificadores numéricos
                "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
                "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
                "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
                "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
            }

            # Parámetros
            costosms = costo_sms  # Costo de cada mensaje SMS
            capacidad_ejecutivos = capacidad_ejecutivos  # Capacidad máxima en términos de tiempo de los ejecutivos

            # Paso 1: Procesar probabilidades y asignar IDs de tratamiento
            logger.info("Procesando datos de probabilidades y asignando IDs de tratamiento...")
            df_probabilities = df_probabilities_treatment.copy()
            df_probabilities['tratamiento_id'] = df_probabilities['Tratamiento'].map(tratamiento_map)
            if df_probabilities['tratamiento_id'].isnull().any():
                logger.warning("Algunos tratamientos no fueron mapeados correctamente a 'tratamiento_id'. Revisar 'tratamiento_map'.")

            # Paso 2: Crear lista de tratamientos y combinar con rut_info
            logger.info("Fusionando probabilidades con información de RUT...")
            df_probabilities['tratamientos'] = df_probabilities[['probabilidad_simular', 'tratamiento_id']].values.tolist()
            grouped_prob = df_probabilities.groupby('categoria_clusterizacion_numerica')['tratamientos'].apply(list).reset_index()

            df_rut_info = df_rut_info.copy()
            df_rut_info = df_rut_info.merge(grouped_prob, on='categoria_clusterizacion_numerica', how='left')
            if df_rut_info['tratamientos'].isnull().any():
                logger.warning("Algunos clusters no tienen tratamientos asignados.")

            # Paso 3: Combinar rut_info con cluster_info
            logger.info("Fusionando información de RUT con información de clusters...")
            df_cluster_info = df_cluster_info.copy()
            df_rut_info = df_rut_info.merge(df_cluster_info, on='categoria_clusterizacion_numerica', how='left')
            if df_rut_info.isnull().any().any():
                logger.warning("Algunas combinaciones de clusters no están completas en rut_info.")

            # Paso 3.5: Agrupar información por cluster en 'rut_info'
            logger.info("Agrupando información por cluster...")
            df_grouped = df_rut_info.groupby('categoria_clusterizacion_numerica').agg({
                'Probabilidad_No_Pago': 'mean',  # Promedio de probabilidad de no pago
                'tratamientos': lambda x: list(x),  # Lista de opciones de tratamiento únicas en cada cluster
                'Monto_Simulado_mean': 'mean',
                'Plazo_Simulado_mean': 'mean',
                'probabilidad_aceptacion_optima': 'mean',
                'tasa_optima': 'mean',
                'rut': 'count'  # Conteo del número de clientes ('rut') en cada cluster
            }).rename(columns={'rut': 'n_clientes'}).reset_index()
            logger.debug(f"Datos agrupados por cluster: {df_grouped.head()}")

            # Paso 4: Calcular 'RC' (Revenue calculado)
            logger.info("Calculando Revenue Calculado (RC)...")
            df_grouped['tasa_optima'] /= 100  # Convertir tasa óptima a decimal
            df_grouped['RC'] = (
                (df_grouped['Plazo_Simulado_mean'] * df_grouped['Monto_Simulado_mean'] * df_grouped['tasa_optima'] *
                ((1 + df_grouped['tasa_optima']) ** df_grouped['Plazo_Simulado_mean'])) /
                (((1 + df_grouped['tasa_optima']) ** df_grouped['Plazo_Simulado_mean']) - 1)
            ) - df_grouped['Monto_Simulado_mean']
            logger.debug(f"RC calculado: {df_grouped['RC'].head()}")

            # -------------------------------
            # Preparación de datos para optimización
            # -------------------------------

            logger.info("Preparando matriz de beneficios para optimización...")
            try:
                # Convertir 'tratamientos' a un arreglo de numpy para mejorar la indexación
                profits = np.array([
                    [
                        row['n_clientes'] * (row['RC'] * (1 - row['Probabilidad_No_Pago']) * row['probabilidad_aceptacion_optima'] * row['tratamientos'][0][t][0]) - 
                        (row['tratamientos'][0][t][1] * costosms)
                        for t in range(8)
                    ]
                    for _, row in df_grouped.iterrows()
                ])
                logger.debug(f"Matriz de beneficios (profits) preparada con forma: {profits.shape}")
            except Exception as e:
                logger.error(f"Error al preparar la matriz de beneficios: {e}")
                del df_grouped, df_rut_info, df_probabilities, df_cluster_info
                gc.collect()
                return {}, 0 # Retornar valores por defecto en caso de error

            # Inicializar el modelo de optimización
            logger.info("Inicializando modelo de optimización con Gurobi...")
            model = Model("Maximizar_Ganancias")
            model.ModelSense = GRB.MAXIMIZE

            # Crear variables de decisión y definir el objetivo
            n_clients, n_treatments = profits.shape
            variables = {}

            logger.info("Creando variables de decisión...")
            for i in range(n_clients):
                variables[i] = {}
                for t in range(n_treatments):
                    if profits[i, t] > 0:
                        variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")
            logger.debug(f"Variables de decisión creadas: {len(variables)} clusters con hasta {n_treatments} tratamientos cada uno.")

            # Definir el objetivo
            model.setObjective(
                quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
            )
            logger.debug("Objetivo del modelo definido.")

            # Restricción: Cada cliente recibe exactamente un tratamiento
            logger.info("Agregando restricción de un tratamiento por cliente...")
            for i in variables:
                model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")
            logger.debug("Restricciones de tratamiento por cliente agregadas.")

            # Restricción de capacidad para ejecutivos
            logger.info("Agregando restricción de capacidad para ejecutivos...")
            model.addConstr(
                quicksum(variables[i][t] * df_grouped.loc[i, 'n_clientes'] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
                name="CapacityConstraint"
            )
            logger.debug("Restricción de capacidad para ejecutivos agregada.")

            # Consistencia de cluster: los clientes dentro del mismo cluster deben recibir el mismo tratamiento
            logger.info("Agregando restricciones de consistencia por cluster...")
            clusters = df_grouped.groupby("categoria_clusterizacion_numerica").indices
            for cluster_id, indices_cluster in clusters.items():
                indices_list = list(indices_cluster)
                leader_index = indices_list[0]
                for t in variables[leader_index]:
                    leader_var = variables[leader_index][t]
                    for i in indices_list[1:]:
                        if t in variables[i]:
                            model.addConstr(variables[i][t] == leader_var, name=f"ClusterConsistency_{cluster_id}_{t}")
            logger.debug("Restricciones de consistencia por cluster agregadas.")

            # Optimizar el modelo
            logger.info("Iniciando optimización con Gurobi...")
            try:
                model.optimize()
            except Exception as e:
                logger.error(f"Error durante la optimización con Gurobi: {e}")
                return {}, 0  # Retornar valores por defecto en caso de error

            # Verificar si la optimización fue exitosa
            if model.Status == GRB.OPTIMAL:
                logger.info("Optimización exitosa. Extrayendo resultados...")

                # Asignar tratamientos por cluster basado en los resultados de la optimización
                resultados_por_cluster = {}
                for cluster_id, indices_cluster in clusters.items():
                    leader_index = list(indices_cluster)[0]
                    for t in variables[leader_index]:
                        if variables[leader_index][t].X > 0.5:
                            resultados_por_cluster[cluster_id] = t + 1
                            break

                # Calcular las ganancias totales
                ganancias_totales = model.ObjVal
                logger.info(f"Ganancias totales: {ganancias_totales:.2f}")

                # Calcular el número de ejecutivos usados y restantes
                executives_used = sum(
                    df_grouped.loc[i, 'n_clientes'] for i in variables for t in variables[i]
                    if t in [5, 6, 7] and variables[i][t].X > 0.5
                )
                executives_remaining = capacidad_ejecutivos - executives_used
                self.executives_remaining = executives_remaining
                logger.info(f"Executives used: {executives_used}")
                logger.info(f"Executives remaining: {executives_remaining}")
            else:
                logger.error("Optimización no alcanzó una solución óptima.")
                resultados_por_cluster = {}
                ganancias_totales = 0
            logger.info("Optimización completada.")

            return resultados_por_cluster, ganancias_totales
        except Exception as e:
            logger.error(f"Error en function_modelo_asignacion_tratamientos: {e}")
            return {}, 0  # Retornar valores por defecto en caso de error global

    def recalculate_metrics(self, df_clusters):
        logger.info("Recalculando métricas...")
        df_clusters = df_clusters[['rut', 'categoria_clusterizacion_numerica']]

        # Fusionar df_clusters con df_simulaciones_e_informacion_de_clientes_ventas_tratamiento
        df_estimar_elasticidad = pd.merge(
            df_clusters,
            self.df_sim_ventas_tratamiento,
            on='rut',
            how='left'
        )
        logger.debug(f"Datos después de la fusión: {df_estimar_elasticidad.head()}")

        dict_elasticidad = self.function_estimar_elasticidad(df_estimar_elasticidad)
        df_elasticidad = dict_elasticidad['df_estimar_elasticidad']
        logger.info(f"total_revenue: {dict_elasticidad['total_revenue']}, total_clientes: {dict_elasticidad['total_clientes']}, total_simulaciones: {dict_elasticidad['total_simulaciones']}, total_creditos: {dict_elasticidad['total_creditos']}")

        df_probabilities = self.function_estimar_respuesta_a_tratamiento(df_elasticidad, self.df_simulaciones_info)
        logger.debug(f"Probabilidades calculadas: {df_probabilities.head()}")

        # Calcular los promedios para 'Monto_Simulado' y 'Plazo_Simulado' por categoría
        df_elasticidad[['Monto_Simulado_mean', 'Plazo_Simulado_mean']] = df_elasticidad.groupby('categoria_clusterizacion_numerica')[['Monto_Simulado', 'Plazo_Simulado']].transform('mean')
        logger.debug(f"Promedios calculados: {df_elasticidad[['Monto_Simulado_mean', 'Plazo_Simulado_mean']].head()}")

        # Reducir tamaño de los DataFrames a las columnas esenciales
        df_estimar_elasticidad_small = df_elasticidad[['categoria_clusterizacion_numerica', 'rut', 'tasa_optima', 'probabilidad_aceptacion_optima', 'Probabilidad_No_Pago', 'Monto_Simulado_mean', 'Plazo_Simulado_mean']]
        df_probabilities_small = df_probabilities[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']]

        # Guardar información por cluster, tratamiento y cliente
        df_cluster_info = df_estimar_elasticidad_small[['categoria_clusterizacion_numerica', 'Monto_Simulado_mean', 'Plazo_Simulado_mean', 'probabilidad_aceptacion_optima', 'tasa_optima']].drop_duplicates()
        df_probabilities_treatment = df_probabilities_small[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']].drop_duplicates()
        df_rut_info = df_estimar_elasticidad_small[['rut', 'categoria_clusterizacion_numerica', 'Probabilidad_No_Pago']].drop_duplicates()

        logger.info("Llamando al modelo de asignación de tratamientos...")
        # Llamar a la función con los DataFrames procesados
        resultados_por_cluster, ganancias_totales = self.function_modelo_asignacion_tratamientos(df_cluster_info, df_probabilities_treatment, df_rut_info, 100, 205000)

        logger.info("Recalculación de métricas completada.")
        return ganancias_totales, len(resultados_por_cluster)

## Entrenamiento

In [47]:
df_info_clientes_rl = df_informacion_de_clientes[['rut', 'Genero', 'Categoria_Digital', 'Elasticidad_Precios', 'Nacionalidad', 'Propension', 'Probabilidad_No_Pago', 'Edad', 'Renta']]

In [49]:
from stable_baselines3 import DQN
import signal

# Function to handle interruptions
def save_model_on_interrupt(model, file_name="dqn_clustering_agent_interrupt"):
    logger.info(f"Guardando el modelo debido a una interrupción como '{file_name}'...")
    model.save(file_name)
    logger.info(f"Modelo guardado exitosamente como '{file_name}'.")

# Try-except block for handling interruptions in a Jupyter Notebook
try:
    # Crear la instancia del entorno
    logger.info("Creando instancia de ClusteringEnv...")
    env = ClusteringEnv(df_info_clientes_rl, df_simulaciones_e_informacion_de_clientes_ventas_tratamiento, df_simulaciones_info)

    # Verificar el entorno (opcional pero recomendado)
    logger.info("Verificando el entorno con check_env...")
    check_env(env, warn=True)

    # Crear y entrenar el agente DQN
    logger.info("Inicializando y entrenando el agente DQN...")
    model = DQN('MlpPolicy', env, verbose=1)

    # Entrenamiento del modelo
    logger.info("Iniciando el entrenamiento del agente DQN...")
    model.learn(total_timesteps=10) 
    logger.info("Entrenamiento del agente DQN completado.")

    # Guardar el modelo entrenado
    model.save("dqn_clustering_agent")
    logger.info("Modelo DQN guardado como 'dqn_clustering_agent'.")

except KeyboardInterrupt:
    logger.warning("Entrenamiento interrumpido por el usuario.")
    file_name="dqn_clustering_agent_interrupt"
    logger.info(f"Guardando el modelo debido a una interrupción como '{file_name}'...")
    model.save(file_name)
    logger.info(f"Modelo guardado exitosamente como '{file_name}'.")


except Exception as e:
    logger.error(f"Se produjo un error durante la ejecución: {str(e)}", exc_info=True)
    save_model_on_interrupt(model, file_name="dqn_clustering_agent_error")


2024-11-27 19:25:32,739 - __main__ - INFO - Variables incluidas en este clustering: ['Elasticidad_Precios_cluster', 'Propension_cluster', 'Renta_cluster']
2024-11-27 19:25:32,741 - __main__ - INFO - Clustering completado. Número de clusters: 13
2024-11-27 19:25:32,743 - __main__ - INFO - Recalculando métricas...
2024-11-27 19:25:36,652 - __main__ - INFO - Estimando elasticidad...
2024-11-27 19:25:41,215 - __main__ - INFO - Cluster 1:
2024-11-27 19:25:41,217 - __main__ - INFO - - Precio Máx. Revenue Esperado = 2.50%
2024-11-27 19:25:41,219 - __main__ - INFO - - Revenue Esperado Máximo = 7,972,981,793.64
2024-11-27 19:25:41,221 - __main__ - INFO - - Número de clientes en el cluster = 61916
2024-11-27 19:25:41,223 - __main__ - INFO - - Número de simulaciones en el cluster = 17261.50
2024-11-27 19:25:41,224 - __main__ - INFO - - Probabilidad de aceptación en el precio óptimo = 0.4006
2024-11-27 19:25:41,226 - __main__ - INFO - - Número esperado de créditos aceptados = 6915
2024-11-27 19:25

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))


2024-11-27 19:26:50,492 - gurobipy - INFO - Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))


2024-11-27 19:26:50,499 - gurobipy - INFO - 


CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]


...

2024-11-27 19:37:20,496 - gurobipy - INFO - 


Optimal solution found (tolerance 1.00e-04)


2024-11-27 19:37:20,500 - gurobipy - INFO - Optimal solution found (tolerance 1.00e-04)


Best objective 9.517722281783e+10, best bound 9.517722281783e+10, gap 0.0000%


2024-11-27 19:37:20,509 - gurobipy - INFO - Best objective 9.517722281783e+10, best bound 9.517722281783e+10, gap 0.0000%
2024-11-27 19:37:20,517 - __main__ - INFO - Optimización exitosa. Extrayendo resultados...
2024-11-27 19:37:20,523 - __main__ - INFO - Ganancias totales: 95177222817.83
2024-11-27 19:37:20,525 - __main__ - INFO - Executives used: 204195
2024-11-27 19:37:20,527 - __main__ - INFO - Executives remaining: 805
2024-11-27 19:37:20,529 - __main__ - INFO - Optimización completada.
2024-11-27 19:37:20,542 - __main__ - INFO - Recalculación de métricas completada.
2024-11-27 19:37:21,806 - __main__ - INFO - Entrenamiento del agente DQN completado.
2024-11-27 19:37:21,890 - __main__ - INFO - Modelo DQN guardado como 'dqn_clustering_agent'.


## Modelo

In [ ]:
# Cargar el modelo entrenado (si ya lo has guardado previamente)
model = DQN.load("dqn_clustering_agent")

# Resetear el entorno
obs, info = env.reset(seed=42)
done = False
truncated = False
total_reward = 0

while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    total_reward += reward
    # Opcional: imprimir la acción y la recompensa
    print(f"Action: {action}, Reward: {reward}")

print(f"Total reward: {total_reward}")


2024-11-27 19:39:18,828 - __main__ - INFO - Reiniciando el entorno...
2024-11-27 19:39:18,831 - __main__ - INFO - Estado reiniciado: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
2024-11-27 19:39:18,928 - __main__ - INFO - Aplicando acción tipo: adjust_splits sobre variable: Renta con parámetros: {'operation': 'move', 'index': 1, 'amount': 1}
2024-11-27 19:39:18,931 - __main__ - WARNING - No se puede ajustar separaciones para 'Renta': no es continua o no está incluida en la clusterización.
2024-11-27 19:39:18,933 - __main__ - INFO - Seleccionando nueva acción: ('adjust_splits', 'Propension', {'operation': 'decrease'})
2024-11-27 19:39:18,936 - __main__ - INFO - Aplicando acción tipo: adjust_splits sobre variable: Propension con parámetros: {'operation': 'decrease'}
2024-11-27 19:39:18,938 - __main__ - WARNING - No se puede ajustar separaciones para 'Propension': no es continua o no está incluida en la clusterización.
2024-11-27 19:39:18,940 - __main__ - INFO - Seleccion

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))


2024-11-27 19:40:43,838 - gurobipy - INFO - Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))


2024-11-27 19:40:43,841 - gurobipy - INFO - 


CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]


...

2024-11-27 19:50:08,657 - gurobipy - INFO - Optimal solution found (tolerance 1.00e-04)


Best objective 1.295323127750e+11, best bound 1.295323127750e+11, gap 0.0000%


2024-11-27 19:50:08,661 - gurobipy - INFO - Best objective 1.295323127750e+11, best bound 1.295323127750e+11, gap 0.0000%
2024-11-27 19:50:08,664 - __main__ - INFO - Optimización exitosa. Extrayendo resultados...
2024-11-27 19:50:08,668 - __main__ - INFO - Ganancias totales: 129532312775.02
2024-11-27 19:50:08,671 - __main__ - INFO - Executives used: 203421
2024-11-27 19:50:08,673 - __main__ - INFO - Executives remaining: 1579
2024-11-27 19:50:08,675 - __main__ - INFO - Optimización completada.
2024-11-27 19:50:08,691 - __main__ - INFO - Recalculación de métricas completada.
2024-11-27 19:50:10,059 - __main__ - INFO - Aplicando acción tipo: adjust_splits sobre variable: Renta con parámetros: {'operation': 'move', 'index': 1, 'amount': 1}
2024-11-27 19:50:10,060 - __main__ - WARNING - No se puede ajustar separaciones para 'Renta': no es continua o no está incluida en la clusterización.
2024-11-27 19:50:10,062 - __main__ - INFO - Seleccionando nueva acción: ('adjust_splits', 'Edad', {'op

Action: 36, Reward: 129036522775.0238


2024-11-27 19:50:10,314 - __main__ - INFO - Variable categórica 'Categoria_Digital' separada en los grupos: ['Cliente no Digital' 'Cliente Digital']
2024-11-27 19:50:10,317 - __main__ - INFO - Procesando variable: Elasticidad_Precios
2024-11-27 19:50:10,399 - __main__ - INFO - Variable categórica 'Elasticidad_Precios' separada en los grupos: ['Alta' 'Baja' 'Media']
2024-11-27 19:50:10,401 - __main__ - INFO - Procesando variable: Nacionalidad
2024-11-27 19:50:10,402 - __main__ - INFO - Procesando variable: Propension
2024-11-27 19:50:10,404 - __main__ - INFO - Procesando variable: Probabilidad_No_Pago
2024-11-27 19:50:10,406 - __main__ - INFO - Procesando variable: Edad
2024-11-27 19:50:10,435 - __main__ - INFO - Cortes para variable 'Edad': [46.0]
2024-11-27 19:50:10,436 - __main__ - INFO - Procesando variable: Renta
2024-11-27 19:50:10,437 - __main__ - INFO - Variables incluidas en clusterización: ['Genero_cluster', 'Categoria_Digital_cluster', 'Elasticidad_Precios_cluster', 'Edad_clu

In [ ]:
import gc

# Paso 1: Definir las variables que deseas mantener
variables_to_keep = [
    'gym',
    'spaces',
    'DQN',
    'check_env',
    'ClusteringEnv',
    'Model',
    'GRB',
    'quicksum',
    'os',
    'datetime',
    'np',
    'pd',
    'sm',
    'df_informacion_de_clientes',
    'df_simulaciones_e_informacion_de_clientes_ventas_tratamiento',
    'env',
    'model',
    'df_simulaciones_info'
    # Añade aquí cualquier otra variable que necesites mantener
]

# Paso 2: Obtener todas las variables en el espacio de nombres global
all_vars = list(globals().keys())

# Paso 3: Identificar las variables que se deben eliminar
# Se excluyen las variables internas que comienzan con '_'
vars_to_delete = [var for var in all_vars if var not in variables_to_keep and not var.startswith('_')]

# Paso 4: Eliminar las variables no necesarias
for var in vars_to_delete:
    try:
        del globals()[var]
        print(f"Variable '{var}' eliminada.")
    except Exception as e:
        print(f"No se pudo eliminar la variable '{var}': {e}")

# Paso 5: Forzar la recolección de basura para liberar la memoria
gc.collect()

print("Variables no necesarias eliminadas y memoria liberada.")
